In [1]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_limit = 11677957  # 2023 carbon emissions limit
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                no_of_km = vehicle['demand'] / num_vehicles
                emissions = num_vehicles * vehicle['carbon_emissions_per_km'] * no_of_km
                total_emissions += emissions
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (within vehicle limits and emissions cap)"""
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        emissions = self.calculate_emissions(solution, size_distance)
        return emissions <= self.total_carbon_limit

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness based on:
        1. Carbon emissions (lower is better)
        2. Meeting demand requirements
        3. TOPSIS score for additional benefits
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = self.calculate_emissions(solution, size_distance)
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Emissions score (lower emissions = higher score)
        emissions_score = 1 / (total_emissions + 1)
        
        return emissions_score + (weighted_topsis * 0.3) - demand_penalty

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if change == -1 and mutated_solution[vehicle_type] > 0:
                        mutated_solution[vehicle_type] += change
                    elif change == 1:
                        mutated_solution[vehicle_type] += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        print(f"Within 2023 limit: {total_carbon_emissions <= self.total_carbon_limit}")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimization with carbon emissions minimization...

Total Carbon Emissions: 15839107.85
Within 2023 limit: False

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     LNG          LNG               9             9   
1   Size S1, Distance D2  Diesel          B20              10            26   
2   Size S1, Distance D2     LNG          LNG              16            26   
3   Size S1, Distance D3  Diesel          B20              12            33   
4   Size S1, Distance D3     LNG          LNG              21            33   
5   Size S1, Distance D4     LNG          LNG               5             5   
6   Size S2, Distance D1     LNG          LNG              10            10   
7   Size S2, Distance D2  Diesel          B20               3            14   
8   Size S2, Distance D2     LNG          LNG              11            14   
9   Size S2, Distance D3     LNG          LNG              

In [3]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_limit = 11677957  # 2023 carbon emissions limit
        self.current_total_emissions = 0  # Track emissions across all optimizations
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                no_of_km = vehicle['demand'] / num_vehicles
                emissions = num_vehicles * vehicle['carbon_emissions_per_km'] * no_of_km
                total_emissions += emissions
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (within vehicle limits and emissions cap)"""
        # Check if we exceed the maximum number of vehicles for this group
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Calculate emissions for this solution
        emissions = self.calculate_emissions(solution, size_distance)
        
        # Check if this solution plus existing emissions would exceed the global limit
        if emissions + self.current_total_emissions > self.total_carbon_limit:
            return False
            
        return True

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness based on:
        1. Carbon emissions (lower is better)
        2. Meeting demand requirements
        3. TOPSIS score for additional benefits
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = self.calculate_emissions(solution, size_distance)
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Emissions score (lower emissions = higher score)
        emissions_score = 1 / (total_emissions + 1)
        
        return emissions_score + (weighted_topsis * 0.3) - demand_penalty

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if change == -1 and mutated_solution[vehicle_type] > 0:
                        mutated_solution[vehicle_type] += change
                    elif change == 1:
                        mutated_solution[vehicle_type] += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        
        # If we couldn't generate any valid initial population, return empty solution
        if not population:
            return {v['vehicle_type']: 0 for v in self.vehicles_by_size_distance[size_distance]}
            
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        self.current_total_emissions = 0  # Reset emissions counter
        
        # Sort the keys to process larger/more polluting groups first
        sorted_keys = sorted(self.vehicles_by_size_distance.keys(), 
                            key=lambda k: sum(v['demand'] * v['carbon_emissions_per_km'] 
                                            for v in self.vehicles_by_size_distance[k]))
        
        for size_distance in sorted_keys:
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            # Calculate emissions for this solution
            solution_emissions = 0
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    solution_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })
            
            # Update the running total of emissions
            self.current_total_emissions += solution_emissions

        print(f"\nTotal Carbon Emissions: {self.current_total_emissions:.2f}")
        print(f"Carbon Limit: {self.total_carbon_limit}")
        print(f"Within 2023 limit: {self.current_total_emissions <= self.total_carbon_limit}")
        
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimization with carbon emissions minimization...

Total Carbon Emissions: 11150394.17
Carbon Limit: 11677957
Within 2023 limit: True

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S4, Distance D4     LNG          LNG               1             1   
1   Size S4, Distance D1     BEV  Electricity               1             1   
2   Size S4, Distance D3     LNG          LNG               2             2   
3   Size S2, Distance D4     LNG          LNG               2             2   
4   Size S3, Distance D4     LNG          LNG               3             3   
5   Size S1, Distance D4     LNG          LNG               5             5   
6   Size S4, Distance D2     LNG          LNG               7             7   
7   Size S2, Distance D3     LNG          LNG               8             8   
8   Size S1, Distance D1     LNG          LNG               9             9   
9   Size S2, Distance D1     LNG     

In [1]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Yearly range (km)': 'Yearly_Range',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Topsis_Score': 'Topsis_Score',
    }
    return df.rename(columns=column_mapping, errors='ignore')

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_limit = 11677957
        self.optimized_solutions = {}
        
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups

    def _calculate_max_vehicles(self) -> Dict:
        return {key: math.ceil(vehicles[0]['demand'] / max(v['yearly_range'] for v in vehicles))
                for key, vehicles in self.vehicles_by_size_distance.items()}

    def calculate_emissions(self, solution: Dict, vehicles: List[Dict]) -> float:
        return sum(
            num_vehicles * vehicle['carbon_emissions_per_km'] * (vehicle['demand'] / num_vehicles)
            for vehicle in vehicles
            for vehicle_type, num_vehicles in solution.items()
            if vehicle_type == vehicle['vehicle_type'] and num_vehicles > 0
        )

    def get_total_emissions(self, current_solution: Dict, size_distance: Tuple) -> float:
        total = self.calculate_emissions(current_solution, self.vehicles_by_size_distance[size_distance])
        return total + sum(
            self.calculate_emissions(solution, self.vehicles_by_size_distance[sd])
            for sd, solution in self.optimized_solutions.items()
            if sd != size_distance
        )

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_vehicles = sum(solution.values())
        if total_vehicles > self.max_vehicles_by_group[size_distance]:
            return False
            
        total_emissions = self.get_total_emissions(solution, size_distance)
        if total_emissions > self.total_carbon_limit:
            return False
            
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        return total_capacity >= vehicles[0]['demand']

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 20) -> List[Dict]:
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        # Create template solution
        template = {v['vehicle_type']: 0 for v in vehicles}
        
        while len(population) < population_size:
            solution = template.copy()
            remaining = max_vehicles
            
            # Distribute vehicles randomly
            for vehicle_type in sorted(template.keys()):
                if remaining <= 0:
                    break
                count = random.randint(0, remaining)
                solution[vehicle_type] = count
                remaining -= count
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def optimize(self, size_distance: Tuple, generations: int = 30) -> Dict:
        population = self.generate_initial_population(size_distance)
        if not population:
            return None
            
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            # Calculate fitness for all solutions
            solutions_with_fitness = [
                (solution, self.calculate_emissions(solution, self.vehicles_by_size_distance[size_distance]))
                for solution in population
                if self.is_valid_solution(solution, size_distance)
            ]
            
            if not solutions_with_fitness:
                break
                
            # Sort by emissions (lower is better)
            solutions_with_fitness.sort(key=lambda x: x[1])
            
            current_best = solutions_with_fitness[0]
            if best_solution is None or current_best[1] < best_fitness:
                best_solution = current_best[0]
                best_fitness = current_best[1]
            
            # Select top performers
            population = [s[0] for s in solutions_with_fitness[:10]]
            
            # Generate new solutions
            while len(population) < 20:
                parent = random.choice(population)
                child = parent.copy()
                
                # Simple mutation
                for vehicle_type in child:
                    if random.random() < 0.3:  # 30% mutation rate
                        child[vehicle_type] = max(0, child[vehicle_type] + random.randint(-1, 1))
                
                if self.is_valid_solution(child, size_distance):
                    population.append(child)
        
        if best_solution:
            self.optimized_solutions[size_distance] = best_solution
        
        return best_solution

    # def get_optimized_results(self) -> pd.DataFrame:
    #     results = []
    #     total_carbon_emissions = 0
        
    #     # Sort by demand to prioritize larger demands
    #     size_distances = sorted(
    #         self.vehicles_by_size_distance.keys(),
    #         key=lambda x: self.vehicles_by_size_distance[x][0]['demand'],
    #         reverse=True
    #     )
        
    #     for size_distance in size_distances:
    #         best_solution = self.optimize(size_distance)
    #         if not best_solution:
    #             continue
                
    #         for vehicle_type, num_vehicles in best_solution.items():
    #             if num_vehicles > 0:
    #                 vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
    #                                  if v['vehicle_type'] == vehicle_type)
                    
    #                 no_of_km = vehicle_data['demand'] / num_vehicles
    #                 emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
    #                 total_carbon_emissions += emissions

    #                 results.append({
    #                     "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
    #                     "Vehicle": vehicle_type,
    #                     "Fuel": vehicle_data['fuel'],
    #                     "no_of_vehicles": num_vehicles,
    #                     "Max Vehicles": self.max_vehicles_by_group[size_distance],
    #                     "Demand": vehicle_data['demand'],
    #                     "Yearly Range": vehicle_data['yearly_range'],
    #                     "Carbon Emissions": round(emissions, 2)
    #                 })

    #     print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
    #     print(f"Carbon Limit: {self.total_carbon_limit}")
    #     print(f"Within 2023 limit: {total_carbon_emissions <= self.total_carbon_limit}")
        
    #     if total_carbon_emissions > self.total_carbon_limit:
    #         raise Exception(f"Solution exceeds carbon limit. Total emissions: {total_carbon_emissions:.2f}")
            
    #     return pd.DataFrame(results)

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        # Sort by demand to prioritize larger demands
        size_distances = sorted(
            self.vehicles_by_size_distance.keys(),
            key=lambda x: self.vehicles_by_size_distance[x][0]['demand'],
            reverse=True
        )
        
        for size_distance in size_distances:
            best_solution = self.optimize(size_distance)
            if not best_solution:
                continue
                
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": self.max_vehicles_by_group[size_distance],
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        print(f"Carbon Limit: {self.total_carbon_limit}")
        print(f"Within 2023 limit: {total_carbon_emissions <= self.total_carbon_limit}")
        
        if total_carbon_emissions > self.total_carbon_limit:
            raise Exception(f"Solution exceeds carbon limit. Total emissions: {total_carbon_emissions:.2f}")
            
        # Convert results to DataFrame and sort by "Allocation"
        results_df = pd.DataFrame(results).sort_values(by="Allocation", ascending=True)
        
        return results_df


def main(csv_path: str):
    print("\nRunning optimized carbon emissions calculation...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(results)
    return results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimized carbon emissions calculation...

Total Carbon Emissions: 8335719.18
Carbon Limit: 11677957
Within 2023 limit: True

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
7   Size S1, Distance D1     BEV  Electricity               9             9   
1   Size S1, Distance D2  Diesel          B20              16            26   
0   Size S1, Distance D3  Diesel          B20              33            33   
10  Size S1, Distance D4  Diesel          B20               5             5   
6   Size S2, Distance D1     BEV  Electricity              10            10   
4   Size S2, Distance D2  Diesel          B20              13            14   
8   Size S2, Distance D3  Diesel          B20               5             8   
12  Size S2, Distance D4     LNG          LNG               1             2   
3   Size S3, Distance D1     BEV  Electricity              30            30   
2   Size S3, Distance D2  Diesel          B20  

In [4]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Yearly range (km)': 'Yearly_Range',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Topsis_Score': 'Topsis_Score',
    }
    return df.rename(columns=column_mapping, errors='ignore')

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.min_vehicles_by_group = self._calculate_min_vehicles()
        self.total_carbon_limit = 11677957
        self.optimized_solutions = {}
        
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups

    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles needed for each size-distance combination"""
        return {key: math.ceil(vehicles[0]['demand'] / max(v['yearly_range'] for v in vehicles))
                for key, vehicles in self.vehicles_by_size_distance.items()}
                
    def _calculate_min_vehicles(self) -> Dict:
        """Calculate minimum vehicles needed for each size-distance combination"""
        return {key: math.ceil(vehicles[0]['demand'] / sum(v['yearly_range'] for v in vehicles))
                for key, vehicles in self.vehicles_by_size_distance.items()}

    def calculate_emissions(self, solution: Dict, vehicles: List[Dict]) -> float:
        """Calculate emissions for a specific solution and vehicles"""
        emissions = 0
        for vehicle in vehicles:
            vehicle_type = vehicle['vehicle_type']
            num_vehicles = solution.get(vehicle_type, 0)
            if num_vehicles > 0:
                no_of_km = vehicle['demand'] / num_vehicles
                emissions += num_vehicles * vehicle['carbon_emissions_per_km'] * no_of_km
        return emissions

    def get_total_emissions(self, current_solution: Dict, size_distance: Tuple) -> float:
        """Calculate total emissions across all solutions including the current one"""
        # Emissions from current solution
        total = self.calculate_emissions(current_solution, self.vehicles_by_size_distance[size_distance])
        
        # Add emissions from all other optimized solutions
        for sd, solution in self.optimized_solutions.items():
            if sd != size_distance:
                total += self.calculate_emissions(solution, self.vehicles_by_size_distance[sd])
                
        return total

    def check_capacity_constraint(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution meets the demand with the allocated vehicles"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        return total_capacity >= vehicles[0]['demand']

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if solution is valid considering all constraints"""
        # Check if total vehicles meet min/max constraints
        total_vehicles = sum(solution.values())
        if total_vehicles < self.min_vehicles_by_group[size_distance] or total_vehicles > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Check if capacity meets demand
        if not self.check_capacity_constraint(solution, size_distance):
            return False
            
        # Check if total emissions stay within limit
        total_emissions = self.get_total_emissions(solution, size_distance)
        return total_emissions <= self.total_carbon_limit

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 30) -> List[Dict]:
        """Generate initial population ensuring all constraints are satisfied"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        min_vehicles = self.min_vehicles_by_group[size_distance]
        demand = vehicles[0]['demand']
        
        # Create template solution
        template = {v['vehicle_type']: 0 for v in vehicles}
        
        attempts = 0
        max_attempts = 1000  # Prevent infinite loops
        
        while len(population) < population_size and attempts < max_attempts:
            attempts += 1
            solution = template.copy()
            
            # First strategy: distribute between min and max vehicles
            target_vehicles = random.randint(min_vehicles, max_vehicles)
            remaining = target_vehicles
            
            # Distribute vehicles randomly
            vehicle_types = list(template.keys())
            random.shuffle(vehicle_types)
            
            for i, vehicle_type in enumerate(vehicle_types):
                # For the last vehicle type, assign all remaining vehicles
                if i == len(vehicle_types) - 1:
                    solution[vehicle_type] = remaining
                else:
                    # Otherwise distribute randomly
                    max_for_type = remaining - (len(vehicle_types) - i - 1)  # Ensure at least 1 for remaining types
                    if max_for_type > 0:
                        solution[vehicle_type] = random.randint(0, max_for_type)
                        remaining -= solution[vehicle_type]
                    
            # Verify the solution meets demand constraint
            if self.check_capacity_constraint(solution, size_distance) and self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        # If we couldn't generate enough solutions, create at least one valid solution
        if not population:
            # Try to create a basic valid solution
            solution = template.copy()
            
            # Sort vehicles by efficiency (yearly range / emissions)
            sorted_vehicles = sorted(vehicles, 
                key=lambda v: v['yearly_range'] / (v['carbon_emissions_per_km'] + 0.0001),
                reverse=True)
                
            # Allocate vehicles starting with most efficient
            remaining_demand = demand
            for vehicle in sorted_vehicles:
                vehicle_type = vehicle['vehicle_type']
                if remaining_demand <= 0:
                    break
                    
                # Calculate how many vehicles of this type we need
                needed = math.ceil(remaining_demand / vehicle['yearly_range'])
                solution[vehicle_type] = needed
                remaining_demand -= needed * vehicle['yearly_range']
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict) -> Dict:
        """Create a child solution by combining two parents"""
        child = {}
        for key in parent1:
            # 50% chance to inherit from each parent
            if random.random() < 0.5:
                child[key] = parent1[key]
            else:
                child[key] = parent2[key]
        return child

    def mutate(self, solution: Dict) -> Dict:
        """Mutate a solution by making small changes"""
        mutated = solution.copy()
        keys = list(mutated.keys())
        
        # Select 1-2 keys to adjust
        num_to_adjust = random.randint(1, min(2, len(keys)))
        keys_to_adjust = random.sample(keys, num_to_adjust)
        
        for key in keys_to_adjust:
            # Add or subtract 1 with 50/50 probability
            change = random.choice([-1, 1])
            mutated[key] = max(0, mutated[key] + change)
        
        return mutated

    def optimize(self, size_distance: Tuple, generations: int = 40) -> Dict:
        """Optimize fleet for a specific size-distance combination"""
        population = self.generate_initial_population(size_distance)
        if not population:
            print(f"Could not generate valid solutions for {size_distance}")
            return None
            
        best_solution = None
        best_fitness = float('inf')  # Lower emissions = better fitness
        
        for gen in range(generations):
            # Calculate fitness (emissions) for all valid solutions
            solutions_with_fitness = []
            for solution in population:
                if self.is_valid_solution(solution, size_distance):
                    emissions = self.calculate_emissions(solution, self.vehicles_by_size_distance[size_distance])
                    solutions_with_fitness.append((solution, emissions))
            
            if not solutions_with_fitness:
                break
                
            # Sort by emissions (lower is better)
            solutions_with_fitness.sort(key=lambda x: x[1])
            
            current_best = solutions_with_fitness[0]
            if best_solution is None or current_best[1] < best_fitness:
                best_solution = current_best[0]
                best_fitness = current_best[1]
            
            # Select top performers
            elite_size = max(1, len(population) // 5)
            elite = [s[0] for s in solutions_with_fitness[:elite_size]]
            
            # Create new population
            new_population = elite.copy()  # Keep the elite solutions
            
            # Generate new solutions through crossover and mutation
            while len(new_population) < len(population):
                # Select two parents using tournament selection
                tournament_size = min(3, len(solutions_with_fitness))
                tournament = random.sample(solutions_with_fitness, tournament_size)
                tournament.sort(key=lambda x: x[1])
                parent1 = tournament[0][0]
                
                tournament = random.sample(solutions_with_fitness, tournament_size)
                tournament.sort(key=lambda x: x[1])
                parent2 = tournament[0][0]
                
                # Crossover
                child = self.crossover(parent1, parent2)
                
                # Mutation (30% chance)
                if random.random() < 0.3:
                    child = self.mutate(child)
                
                if self.is_valid_solution(child, size_distance):
                    new_population.append(child)
            
            population = new_population[:len(population)]
        
        if best_solution:
            self.optimized_solutions[size_distance] = best_solution
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization for all size-distance combinations and return results"""
        results = []
        total_carbon_emissions = 0
        
        # Sort by demand to prioritize larger demands
        size_distances = sorted(
            self.vehicles_by_size_distance.keys(),
            key=lambda x: self.vehicles_by_size_distance[x][0]['demand'],
            reverse=True
        )
        
        for size_distance in size_distances:
            best_solution = self.optimize(size_distance)
            if not best_solution:
                print(f"Warning: Could not find a valid solution for {size_distance}")
                continue
                
            # Verify the solution meets the max vehicles constraint
            total_vehicles = sum(best_solution.values())
            max_vehicles = self.max_vehicles_by_group[size_distance]
            if total_vehicles > max_vehicles:
                print(f"Warning: Solution for {size_distance} uses {total_vehicles} vehicles, "
                      f"exceeding the maximum of {max_vehicles}")
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": self.max_vehicles_by_group[size_distance],
                        "Total Vehicles Used": total_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        print(f"Carbon Limit: {self.total_carbon_limit}")
        print(f"Within 2023 limit: {total_carbon_emissions <= self.total_carbon_limit}")
        
        if total_carbon_emissions > self.total_carbon_limit:
            raise Exception(f"Solution exceeds carbon limit. Total emissions: {total_carbon_emissions:.2f}")
            
        # Convert results to DataFrame and sort by "Allocation"
        results_df = pd.DataFrame(results).sort_values(by="Allocation", ascending=True)
        
        return results_df


def main(csv_path: str):
    """Main function to run the fleet optimization"""
    print("\nRunning optimized carbon emissions calculation...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(results)
    return results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimized carbon emissions calculation...

Total Carbon Emissions: 8906770.68
Carbon Limit: 11677957
Within 2023 limit: True

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
11  Size S1, Distance D1     LNG          LNG               4             9   
10  Size S1, Distance D1     BEV  Electricity               1             9   
1   Size S1, Distance D2     LNG          LNG              14            26   
0   Size S1, Distance D3     LNG          LNG              19            33   
14  Size S1, Distance D4     LNG          LNG               4             5   
8   Size S2, Distance D1     BEV  Electricity               2            10   
9   Size S2, Distance D1     LNG          LNG               7            10   
6   Size S2, Distance D2     LNG          LNG               9            14   
12  Size S2, Distance D3     LNG          LNG               6             8   
16  Size S2, Distance D4     LNG          LNG  

In [3]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Yearly range (km)': 'Yearly_Range',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Topsis_Score': 'Topsis_Score',
    }
    return df.rename(columns=column_mapping, errors='ignore')

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_limit = 11677957
        self.optimized_solutions = {}
        
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups

    def _calculate_max_vehicles(self) -> Dict:
        return {key: math.ceil(vehicles[0]['demand'] / max(v['yearly_range'] for v in vehicles))
                for key, vehicles in self.vehicles_by_size_distance.items()}

    def calculate_emissions(self, solution: Dict, vehicles: List[Dict]) -> float:
        return sum(
            num_vehicles * vehicle['carbon_emissions_per_km'] * (vehicle['demand'] / num_vehicles)
            for vehicle in vehicles
            for vehicle_type, num_vehicles in solution.items()
            if vehicle_type == vehicle['vehicle_type'] and num_vehicles > 0
        )

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_vehicles = sum(solution.values())
        if total_vehicles != self.max_vehicles_by_group[size_distance]:
            return False
            
        total_emissions = self.calculate_emissions(solution, vehicles)
        if total_emissions > self.total_carbon_limit:
            return False
            
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        return total_capacity >= vehicles[0]['demand']

    def optimize(self, size_distance: Tuple) -> Dict:
        max_vehicles = self.max_vehicles_by_group[size_distance]
        vehicles = self.vehicles_by_size_distance[size_distance]
        solution = {v['vehicle_type']: 0 for v in vehicles}
        
        # Distribute vehicles among options to meet max_vehicles
        remaining = max_vehicles
        for vehicle in sorted(vehicles, key=lambda v: v['topsis_score'], reverse=True):
            if remaining <= 0:
                break
            allocation = min(remaining, math.ceil(vehicle['demand'] / vehicle['yearly_range']))
            solution[vehicle['vehicle_type']] = allocation
            remaining -= allocation
        
        if self.is_valid_solution(solution, size_distance):
            self.optimized_solutions[size_distance] = solution
            return solution
        return None

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        size_distances = sorted(
            self.vehicles_by_size_distance.keys(),
            key=lambda x: self.vehicles_by_size_distance[x][0]['demand'],
            reverse=True
        )
        
        for size_distance in size_distances:
            best_solution = self.optimize(size_distance)
            if not best_solution:
                continue
                
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": self.max_vehicles_by_group[size_distance],
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        print(f"Carbon Limit: {self.total_carbon_limit}")
        print(f"Within 2023 limit: {total_carbon_emissions <= self.total_carbon_limit}")
        
        if total_carbon_emissions > self.total_carbon_limit:
            raise Exception(f"Solution exceeds carbon limit. Total emissions: {total_carbon_emissions:.2f}")
        
        return pd.DataFrame(results).sort_values(by="Allocation", ascending=True)

def main(csv_path: str):
    print("\nRunning optimized carbon emissions calculation...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(results)
    return results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)



Running optimized carbon emissions calculation...

Total Carbon Emissions: 10592321.35
Carbon Limit: 11677957
Within 2023 limit: True

Optimized Fleet Allocation:
              Allocation Vehicle Fuel  no_of_vehicles  Max Vehicles   Demand  \
7   Size S1, Distance D1     LNG  LNG               9             9   869181   
1   Size S1, Distance D2  Diesel  B20              26            26  2597094   
0   Size S1, Distance D3  Diesel  B20              33            33  3292011   
10  Size S1, Distance D4  Diesel  B20               5             5   414315   
6   Size S2, Distance D1     LNG  LNG              10            10   995694   
4   Size S2, Distance D2  Diesel  B20              14            14  1383196   
8   Size S2, Distance D3  Diesel  B20               8             8   778008   
12  Size S2, Distance D4  Diesel  B20               2             2   133677   
3   Size S3, Distance D1     LNG  LNG              30            30  2183475   
2   Size S3, Distance D2  Diesel  B2

In [1]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Yearly range (km)': 'Yearly_Range',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Topsis_Score': 'Topsis_Score',
    }
    return df.rename(columns=column_mapping, errors='ignore')

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_limit = 11677957
        self.optimized_solutions = {}
        
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups

    def _calculate_max_vehicles(self) -> Dict:
        return {key: math.ceil(vehicles[0]['demand'] / max(v['yearly_range'] for v in vehicles))
                for key, vehicles in self.vehicles_by_size_distance.items()}

    def calculate_emissions(self, solution: Dict, vehicles: List[Dict]) -> float:
        return sum(
            num_vehicles * vehicle['carbon_emissions_per_km'] * (vehicle['demand'] / num_vehicles)
            for vehicle in vehicles
            for vehicle_type, num_vehicles in solution.items()
            if vehicle_type == vehicle['vehicle_type'] and num_vehicles > 0
        )

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_vehicles = sum(solution.values())
        if total_vehicles != self.max_vehicles_by_group[size_distance]:
            return False
            
        total_emissions = self.calculate_emissions(solution, vehicles)
        if total_emissions > self.total_carbon_limit:
            return False
            
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        return total_capacity >= vehicles[0]['demand']

    def optimize(self, size_distance: Tuple) -> Dict:
        max_vehicles = self.max_vehicles_by_group[size_distance]
        vehicles = self.vehicles_by_size_distance[size_distance]
        solution = {v['vehicle_type']: 0 for v in vehicles}
        
        # Distribute vehicles among options to meet max_vehicles
        remaining = max_vehicles
        for vehicle in sorted(vehicles, key=lambda v: v['topsis_score'], reverse=True):
            if remaining <= 0:
                break
            allocation = min(remaining, math.ceil(vehicle['demand'] / vehicle['yearly_range']))
            solution[vehicle['vehicle_type']] = allocation
            remaining -= allocation
        
        if self.is_valid_solution(solution, size_distance):
            self.optimized_solutions[size_distance] = solution
            return solution
        return None

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        size_distances = sorted(
            self.vehicles_by_size_distance.keys(),
            key=lambda x: self.vehicles_by_size_distance[x][0]['demand'],
            reverse=True
        )
        
        for size_distance in size_distances:
            best_solution = self.optimize(size_distance)
            if not best_solution:
                continue
                
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": self.max_vehicles_by_group[size_distance],
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        print(f"Carbon Limit: {self.total_carbon_limit}")
        print(f"Within 2023 limit: {total_carbon_emissions <= self.total_carbon_limit}")
        
        if total_carbon_emissions > self.total_carbon_limit:
            raise Exception(f"Solution exceeds carbon limit. Total emissions: {total_carbon_emissions:.2f}")
        
        return pd.DataFrame(results).sort_values(by="Allocation", ascending=True)

def main(csv_path: str):
    print("\nRunning optimized carbon emissions calculation...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(results)
    return results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2024.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)



Running optimized carbon emissions calculation...

Total Carbon Emissions: 10834409.12
Carbon Limit: 11677957
Within 2023 limit: True

Optimized Fleet Allocation:
              Allocation Vehicle Fuel  no_of_vehicles  Max Vehicles   Demand  \
7   Size S1, Distance D1     LNG  LNG               9             9   877242   
1   Size S1, Distance D2  Diesel  B20              27            27  2716195   
0   Size S1, Distance D3  Diesel  B20              33            33  3336604   
10  Size S1, Distance D4  Diesel  B20               5             5   419981   
5   Size S2, Distance D1     LNG  LNG              10            10  1043209   
4   Size S2, Distance D2  Diesel  B20              14            14  1391987   
8   Size S2, Distance D3  Diesel  B20               8             8   807076   
12  Size S2, Distance D4  Diesel  B20               2             2   133712   
3   Size S3, Distance D1     LNG  LNG              31            31  2239440   
2   Size S3, Distance D2  Diesel  B2

In [ ]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Carbon_emissions_per_km': 'carbon_emissions',  # New column
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Emissions': 'Total_Emissions',  # New column
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    print("Columns after renaming:", df.columns)
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions': row['Carbon_Emissions'],  # Changed from cost
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 1000

    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles

    def calculate_total_emissions(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total carbon emissions for a vehicle type"""
        yearly_emissions = vehicle['carbon_emissions'] * vehicle['yearly_range']
        return num_vehicles * yearly_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if solution is within vehicle limits"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution"""
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness based on:
        1. Carbon emissions (primary objective to minimize)
        2. Meeting demand requirements
        3. TOPSIS score
        4. Utilization
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        total_capacity = 0
        weighted_topsis = 0
        utilization_scores = []
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                total_emissions += self.calculate_total_emissions(num_vehicles, vehicle)
                total_capacity += num_vehicles * vehicle['yearly_range']
                weighted_topsis += num_vehicles * vehicle['topsis_score']
                
                utilization = self.calculate_utilization(
                    vehicle['demand'],
                    num_vehicles,
                    vehicle['yearly_range']
                )
                utilization_scores.append(utilization)
        
        avg_utilization = np.mean(utilization_scores) if utilization_scores else 0
        
        # Demand fulfillment penalty
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Normalize emissions (lower is better)
        emissions_score = 1 / (total_emissions + 1)
        
        # Combined score with weights favoring emissions reduction
        return (
            emissions_score * 0.4 +  # Increased weight for emissions
            weighted_topsis * 0.2 +
            avg_utilization * 0.2 -
            demand_penalty
        )

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_emissions = self.calculate_total_emissions(num_vehicles, vehicle_data)

                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon_Emissions": round(total_emissions, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2)
                    })

        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization for minimizing carbon emissions...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon_2023.csv", index=False)


Running optimization for minimizing carbon emissions...
Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost ($)', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')


KeyError: 'Carbon_Emissions'

In [2]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Carbon_emissions_per_km': 'carbon_emissions',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Emissions': 'Total_Emissions',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions': row['carbon_emissions'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles
    
    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 1000
    
    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles
    
    def calculate_total_carbon_emissions(self, num_vehicles: int, vehicle: Dict) -> float:
        """Calculate total carbon emissions"""
        if num_vehicles == 0:
            return 0
        no_of_km = vehicle['demand'] / num_vehicles
        total_carbon = vehicle['carbon_emissions'] * no_of_km * num_vehicles
        return total_carbon
    
    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if solution is within vehicle limits"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]
    
    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run optimization (dummy implementation)"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        solution = {v['vehicle_type']: random.randint(0, self.max_vehicles_by_group[size_distance]) for v in vehicles}
        return solution
    
    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] if v['vehicle_type'] == vehicle_type)
                    total_emissions = self.calculate_total_carbon_emissions(num_vehicles, vehicle_data)
                    total_carbon_emissions += total_emissions
                    
                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon_Emissions": round(total_emissions, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2)
                    })
        
        print(f"Total Carbon Emissions: {round(total_carbon_emissions, 2)} kg")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your CSV file path
    results_df = main(csv_path)


KeyError: 'carbon_emissions'

In [2]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        # Removed the total_carbon_limit constraint
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                no_of_km = vehicle['demand'] / num_vehicles
                emissions = num_vehicles * vehicle['carbon_emissions_per_km'] * no_of_km
                total_emissions += emissions
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (only checking vehicle limits now)"""
        # Only checking if we don't exceed max vehicles per group
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness based on:
        1. Carbon emissions (lower is better) - primary focus
        2. Meeting demand requirements
        3. TOPSIS score for additional benefits
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = self.calculate_emissions(solution, size_distance)
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        # Penalize solutions that don't meet demand
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        # Emissions score (lower emissions = higher score)
        # Increased weight on emissions by multiplying by 10
        emissions_score = 10 / (total_emissions + 1)
        
        return emissions_score + (weighted_topsis * 0.1) - demand_penalty

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        """Mutate a solution"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if change == -1 and mutated_solution[vehicle_type] > 0:
                        mutated_solution[vehicle_type] += change
                    elif change == 1:
                        mutated_solution[vehicle_type] += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        # Removed the carbon limit check
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimization with carbon emissions minimization...

Total Carbon Emissions: 16518600.03

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     LNG          LNG               9             9   
1   Size S1, Distance D2  Diesel          B20              11            26   
2   Size S1, Distance D2     LNG          LNG              15            26   
3   Size S1, Distance D3  Diesel          B20              13            33   
4   Size S1, Distance D3     LNG          LNG              20            33   
5   Size S1, Distance D4     LNG          LNG               5             5   
6   Size S2, Distance D1  Diesel          B20               6            10   
7   Size S2, Distance D1     LNG          LNG               4            10   
8   Size S2, Distance D2  Diesel          B20               7            14   
9   Size S2, Distance D2     LNG          LNG               7            14   
10  S

In [3]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                no_of_km = min(vehicle['yearly_range'] * num_vehicles, vehicle['demand'])
                emissions = vehicle['carbon_emissions_per_km'] * no_of_km
                total_emissions += emissions
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (meets vehicle limits and demand)"""
        # Check if we don't exceed max vehicles per group
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Ensure demand is met
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        demand = vehicles[0]['demand']
        
        return total_capacity >= demand

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness with strong preference for BEVs and zero-emission vehicles
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = self.calculate_emissions(solution, size_distance)
        
        # Calculate BEV ratio (higher is better)
        total_vehicles = sum(solution.values())
        if total_vehicles == 0:
            bev_ratio = 0
        else:
            bev_count = sum(solution[v['vehicle_type']] for v in vehicles if v['fuel'] == 'Electricity')
            bev_ratio = bev_count / total_vehicles
        
        # Very strong preference for lower emissions and BEVs
        emissions_score = 100 / (total_emissions + 1)
        bev_bonus = bev_ratio * 50  # Strong bonus for electric vehicles
        
        # Include a small TOPSIS component
        weighted_topsis = sum(solution[v['vehicle_type']] * v['topsis_score'] for v in vehicles) * 0.05
        
        return emissions_score + bev_bonus + weighted_topsis

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 100) -> List[Dict]:
        """Generate initial population with BEV preference"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        # Find BEV vehicles in this group if any
        bev_vehicles = [v['vehicle_type'] for v in vehicles if v['fuel'] == 'Electricity']
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            
            # First try to assign BEV vehicles with higher probability
            if bev_vehicles:
                for bev in bev_vehicles:
                    # Higher chance to assign BEVs
                    max_bevs = min(max_vehicles, 5)  # Limit to avoid too many at once
                    solution[bev] = random.randint(0, max_bevs)
            
            # Then fill remaining with other vehicles
            remaining_vehicles = max_vehicles - sum(solution.values())
            remaining_types = [v['vehicle_type'] for v in vehicles if v['vehicle_type'] not in bev_vehicles]
            
            if remaining_vehicles > 0 and remaining_types:
                while remaining_vehicles > 0 and remaining_types:
                    vehicle_type = random.choice(remaining_types)
                    if random.random() < 0.5:
                        solution[vehicle_type] += 1
                        remaining_vehicles -= 1
                    else:
                        remaining_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.2) -> Dict:
        """Mutate a solution with BEV preference"""
        attempts = 0
        max_attempts = 10
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        bev_types = [v['vehicle_type'] for v in vehicles if v['fuel'] == 'Electricity']
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            
            # Higher mutation rate for BEVs to increase their numbers
            for vehicle_type in mutated_solution.keys():
                # Higher mutation rate for BEVs (to increase)
                if vehicle_type in bev_types:
                    if random.random() < mutation_rate * 2:  # Double mutation rate for BEVs
                        mutated_solution[vehicle_type] += 1  # Always increase BEVs
                # Regular mutation for other vehicle types
                elif random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if change == -1 and mutated_solution[vehicle_type] > 0:
                        mutated_solution[vehicle_type] += change
                    elif change == 1:
                        mutated_solution[vehicle_type] += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 150) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            # Check if any electric vehicles are available for this size-distance
            has_bevs = any(v['fuel'] == 'Electricity' for v in self.vehicles_by_size_distance[size_distance])
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = min(vehicle_data['yearly_range'] * num_vehicles, vehicle_data['demand'])
                    emissions = vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2),
                        "BEVs Available": has_bevs
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with carbon emissions minimization and BEV preference...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimization with carbon emissions minimization and BEV preference...

Total Carbon Emissions: 5392102.12

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     BEV  Electricity               9             9   
1   Size S1, Distance D2     LNG          LNG              26            26   
2   Size S1, Distance D3     LNG          LNG              33            33   
3   Size S1, Distance D4     LNG          LNG               5             5   
4   Size S2, Distance D1     BEV  Electricity              10            10   
5   Size S2, Distance D2     LNG          LNG              14            14   
6   Size S2, Distance D3     LNG          LNG               8             8   
7   Size S2, Distance D4     LNG          LNG               2             2   
8   Size S3, Distance D1     BEV  Electricity              30            30   
9   Size S3, Distance D2     LNG          LNG              34     

In [1]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                no_of_km = min(vehicle['yearly_range'] * num_vehicles, vehicle['demand'])
                emissions = vehicle['carbon_emissions_per_km'] * no_of_km
                total_emissions += emissions
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (meets vehicle limits and demand)"""
        # Check if we don't exceed max vehicles per group
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Ensure demand is met
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        demand = vehicles[0]['demand']
        
        return total_capacity >= demand

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness based on emissions and TOPSIS score without BEV preference
        """
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
            
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = self.calculate_emissions(solution, size_distance)
        
        # Primary focus on emissions reduction
        emissions_score = 100 / (total_emissions + 1)
        
        # Include TOPSIS component for balanced optimization
        weighted_topsis = sum(solution[v['vehicle_type']] * v['topsis_score'] for v in vehicles) * 0.1
        
        return emissions_score + weighted_topsis

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 100) -> List[Dict]:
        """Generate initial population without BEV preference"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            
            # Randomly assign vehicles without preference
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = {}
            child2 = {}
            
            for i, vehicle_type in enumerate(parent1.keys()):
                if i < crossover_point:
                    child1[vehicle_type] = parent1[vehicle_type]
                    child2[vehicle_type] = parent2[vehicle_type]
                else:
                    child1[vehicle_type] = parent2[vehicle_type]
                    child2[vehicle_type] = parent1[vehicle_type]
            
            if (self.is_valid_solution(child1, size_distance) and 
                self.is_valid_solution(child2, size_distance)):
                return child1, child2
            
            attempts += 1
        
        return parent1.copy(), parent2.copy()

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.2) -> Dict:
        """Mutate a solution without BEV preference"""
        attempts = 0
        max_attempts = 10
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            
            # Apply mutation equally to all vehicle types
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if change == -1 and mutated_solution[vehicle_type] > 0:
                        mutated_solution[vehicle_type] += change
                    elif change == 1:
                        mutated_solution[vehicle_type] += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 150) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            # Record vehicle types without distinguishing fuel types
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    
                    no_of_km = min(vehicle_data['yearly_range'] * num_vehicles, vehicle_data['demand'])
                    emissions = vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Carbon Emissions": round(emissions, 2)
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimization with carbon emissions minimization...

Total Carbon Emissions: 7053548.12

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     LNG          LNG               9             9   
1   Size S1, Distance D2     LNG          LNG              26            26   
2   Size S1, Distance D3     LNG          LNG              33            33   
3   Size S1, Distance D4     LNG          LNG               5             5   
4   Size S2, Distance D1     LNG          LNG              10            10   
5   Size S2, Distance D2     LNG          LNG              14            14   
6   Size S2, Distance D3     LNG          LNG               8             8   
7   Size S2, Distance D4     LNG          LNG               2             2   
8   Size S3, Distance D1     LNG          LNG              30            30   
9   Size S3, Distance D2     LNG          LNG              34            34   
10  Si

In [ ]:
# DONE DONE DONE !!!!!!

In [2]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return (demand / num_vehicles) / yearly_range * 100

    def calculate_demand_fulfillment(self, num_vehicles: int, max_vehicles: int) -> float:
        """Calculate demand fulfillment by fuel type"""
        if max_vehicles == 0:
            return 0
        return num_vehicles / max_vehicles

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        # Calculate how much of the demand is satisfied by each vehicle type
        demand = vehicles[0]['demand']
        remaining_demand = demand
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            if num_vehicles > 0:
                # Calculate how much distance this vehicle type can cover
                vehicle_capacity = num_vehicles * vehicle['yearly_range']
                assigned_demand = min(vehicle_capacity, remaining_demand)
                remaining_demand -= assigned_demand
                
                # Calculate emissions for this portion of demand
                emissions = vehicle['carbon_emissions_per_km'] * assigned_demand
                total_emissions += emissions
        
        # Apply a penalty if demand is not fully met
        if remaining_demand > 0:
            return float('inf')
            
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (meets vehicle limits and demand)"""
        # Check if we don't exceed max vehicles per group
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Ensure demand is met
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        demand = vehicles[0]['demand']
        
        return total_capacity >= demand

    # def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
    #     """
    #     Calculate fitness primarily based on carbon emissions
    #     Lower emissions = higher fitness score
    #     """
    #     if not self.is_valid_solution(solution, size_distance):
    #         return float('-inf')
            
    #     # Calculate emissions - the core focus of optimization
    #     total_emissions = self.calculate_emissions(solution, size_distance)
        
    #     # If emissions are infinite (demand not met), return -inf
    #     if total_emissions == float('inf'):
    #         return float('-inf')
        
    #     # Inverse relationship: lower emissions = higher fitness
    #     # Using a very large factor (1000) to make carbon emissions the dominant factor
    #     emissions_score = 1000000 / (total_emissions + 1)
        
    #     # Add a small component for efficiency (fewer vehicles is better)
    #     vehicles_count = sum(solution.values())
    #     if vehicles_count > 0:
    #         efficiency_score = self.max_vehicles_by_group[size_distance] / vehicles_count
    #     else:
    #         efficiency_score = 0
            
    #     # Return combined score
    #     return emissions_score * 0.95 + efficiency_score * 0.05

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness of a solution based on:
        1. TOPSIS score
        2. Meeting demand requirements
        3. Carbon emissions
        4. Penalty for invalid solutions
        """
        # Check if solution is valid first
        if not self.is_valid_solution(solution, size_distance):
            # Calculate deficit to guide evolution
            deficit = self.demand_deficit(solution, size_distance)
            return float('-inf') + (-1000 * deficit)  # Return negative infinity with gradient
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        
        # Calculate TOPSIS component
        weighted_topsis = 0
        total_vehicles = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            weighted_topsis += num_vehicles * vehicle['topsis_score']
            total_vehicles += num_vehicles
        
        # Normalize weighted TOPSIS by number of vehicles
        if total_vehicles > 0:
            weighted_topsis = weighted_topsis / total_vehicles
        
        # Calculate emissions component
        emissions = self.calculate_emissions(solution, size_distance)
        
        # If emissions are infinite (demand not met), this shouldn't happen but handle it
        if emissions == float('inf'):
            return float('-inf')
        
        # Inverse relationship: lower emissions = higher fitness
        emissions_score = 1000000 / (emissions + 1)
        
        # Return combined score with emphasis on TOPSIS
        return weighted_topsis  + emissions_score 

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 150) -> List[Dict]:
        """Generate initial population of fleet combinations with TOPSIS score weighting"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        demand = vehicles[0]['demand']
        
        # Calculate normalized TOPSIS scores for biasing initial population
        topsis_scores = [v['topsis_score'] for v in vehicles]
        total_topsis = sum(topsis_scores)
        normalized_topsis = [score/total_topsis for score in topsis_scores] if total_topsis > 0 else [1/len(topsis_scores)]*len(topsis_scores)
        
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        # Create solutions with TOPSIS weighting
        attempts = 0
        max_attempts = population_size * 10  # Limit attempts to avoid infinite loops
        
        while len(population) < population_size and attempts < max_attempts:
            attempts += 1
            
            # Create a new solution
            solution = {v_type: 0 for v_type in vehicle_types}
            remaining_vehicles = max_vehicles
            
            # First ensure demand is met
            remaining_demand = demand
            while remaining_demand > 0 and sum(solution.values()) < remaining_vehicles:
                # Select vehicle type with probability proportional to TOPSIS score
                selected_index = random.choices(range(len(vehicle_types)), weights=normalized_topsis, k=1)[0]
                selected_type = vehicle_types[selected_index]
                
                # Add one of this type
                solution[selected_type] += 1
                remaining_demand -= vehicle_ranges[selected_type]
                
                # Stop early with some probability to create diverse solutions
                if random.random() < 0.1 and remaining_demand <= 0:
                    break
            
            # Check if valid and not duplicate
            if self.is_valid_solution(solution, size_distance) and solution not in population:
                population.append(solution)
        
        # If we couldn't generate enough solutions, create some backup solutions
        if len(population) < population_size:
            # Add a TOPSIS-based greedy solution
            topsis_sorted_vehicles = sorted(vehicles, key=lambda v: v['topsis_score'], reverse=True)
            topsis_solution = {v_type: 0 for v_type in vehicle_types}
            
            remaining_demand = demand
            for vehicle in topsis_sorted_vehicles:
                vehicle_type = vehicle['vehicle_type']
                while remaining_demand > 0 and sum(topsis_solution.values()) < max_vehicles:
                    topsis_solution[vehicle_type] += 1
                    remaining_demand -= vehicle['yearly_range']
            
            if self.is_valid_solution(topsis_solution, size_distance) and topsis_solution not in population:
                population.append(topsis_solution)
        
        # Fill remaining slots with mutations of existing solutions
        while len(population) < population_size:
            if not population:  # Safety check
                base_solution = self._create_basic_valid_solution(size_distance)
                population.append(base_solution)
            
            # Take a random solution and mutate it
            base = random.choice(population)
            mutated = self.mutate(base, size_distance, 0.3)
            
            if self.is_valid_solution(mutated, size_distance):
                population.append(mutated)
        
        return population


    # def generate_initial_population(self, size_distance: Tuple, population_size: int = 100) -> List[Dict]:
    #     """Generate diverse initial population"""
    #     population = []
    #     vehicles = self.vehicles_by_size_distance[size_distance]
    #     max_vehicles = self.max_vehicles_by_group[size_distance]
        
    #     # Get all vehicle types
    #     vehicle_types = [v['vehicle_type'] for v in vehicles]
        
    #     # Create solutions with diversity
    #     while len(population) < population_size:
    #         # Try different combinations - some with few vehicles, some with many
    #         solution = {v_type: 0 for v_type in vehicle_types}
            
    #         # Random approach to vehicle assignment
    #         remaining_slots = max_vehicles
    #         available_types = vehicle_types.copy()
            
    #         while remaining_slots > 0 and available_types:
    #             # Randomly select a vehicle type
    #             vehicle_type = random.choice(available_types)
                
    #             # Assign a random number of this vehicle type (1 to remaining)
    #             if remaining_slots > 1:
    #                 count = random.randint(1, min(remaining_slots, max_vehicles // 2))
    #             else:
    #                 count = 1
                    
    #             solution[vehicle_type] += count
    #             remaining_slots -= count
                
    #             # Sometimes remove this type from further consideration
    #             if random.random() < 0.3:
    #                 available_types.remove(vehicle_type)
            
    #         # Check if valid and add to population
    #         if self.is_valid_solution(solution, size_distance):
    #             population.append(solution)
        
    #     return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions"""
        attempts = 0
        max_attempts = 15
        
        while attempts < max_attempts:
            # Use two-point crossover for better diversity
            vehicle_types = list(parent1.keys())
            if len(vehicle_types) < 3:
                # For small number of types, use single point crossover
                crossover_point = random.randint(1, len(vehicle_types) - 1)
                
                child1 = {}
                child2 = {}
                
                for i, vehicle_type in enumerate(vehicle_types):
                    if i < crossover_point:
                        child1[vehicle_type] = parent1[vehicle_type]
                        child2[vehicle_type] = parent2[vehicle_type]
                    else:
                        child1[vehicle_type] = parent2[vehicle_type]
                        child2[vehicle_type] = parent1[vehicle_type]
            else:
                # For more types, use two-point crossover
                points = sorted(random.sample(range(1, len(vehicle_types)), 2))
                
                child1 = {}
                child2 = {}
                
                for i, vehicle_type in enumerate(vehicle_types):
                    if i < points[0] or i >= points[1]:
                        child1[vehicle_type] = parent1[vehicle_type]
                        child2[vehicle_type] = parent2[vehicle_type]
                    else:
                        child1[vehicle_type] = parent2[vehicle_type]
                        child2[vehicle_type] = parent1[vehicle_type]
            
            # Check validity
            valid1 = self.is_valid_solution(child1, size_distance)
            valid2 = self.is_valid_solution(child2, size_distance)
            
            # Return if both valid
            if valid1 and valid2:
                return child1, child2
            
            # If only one is valid, try to fix the other
            if valid1 and not valid2:
                child2 = self._repair_solution(child2, size_distance)
                if self.is_valid_solution(child2, size_distance):
                    return child1, child2
            elif valid2 and not valid1:
                child1 = self._repair_solution(child1, size_distance)
                if self.is_valid_solution(child1, size_distance):
                    return child1, child2
                    
            attempts += 1
        
        # If no valid combination found, return the original parents
        return parent1.copy(), parent2.copy()
    
    def _repair_solution(self, solution: Dict, size_distance: Tuple) -> Dict:
        """Attempt to repair an invalid solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        demand = vehicles[0]['demand']
        
        # Calculate current capacity
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        
        # If we exceed max vehicles, reduce
        max_vehicles = self.max_vehicles_by_group[size_distance]
        total_vehicles = sum(solution.values())
        
        if total_vehicles > max_vehicles:
            # Remove vehicles randomly until we're under the limit
            while sum(solution.values()) > max_vehicles:
                # Pick a vehicle type that has at least one vehicle
                vehicle_types = [vt for vt in solution.keys() if solution[vt] > 0]
                if not vehicle_types:
                    break
                vehicle_type = random.choice(vehicle_types)
                solution[vehicle_type] -= 1
        
        # If we don't meet demand, add vehicles
        if total_capacity < demand:
            # Sort vehicles by emissions per km (ascending)
            sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
            
            # Try to add low-emission vehicles first
            for vehicle in sorted_vehicles:
                vehicle_type = vehicle['vehicle_type']
                while (total_capacity < demand and 
                       sum(solution.values()) < max_vehicles):
                    solution[vehicle_type] += 1
                    total_capacity += vehicle['yearly_range']
                    
                if total_capacity >= demand:
                    break
        
        return solution

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.2) -> Dict:
        """Mutate a solution with focus on emissions reduction"""
        attempts = 0
        max_attempts = 15
        vehicles = self.vehicles_by_size_distance[size_distance]
        
        # Sort vehicles by carbon emissions (lowest first)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            mutation_happened = False
            
            # First approach: randomly adjust vehicle counts
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    mutation_happened = True
                    change = random.choice([-1, 1])
                    if change == -1 and mutated_solution[vehicle_type] > 0:
                        mutated_solution[vehicle_type] += change
                    elif change == 1:
                        mutated_solution[vehicle_type] += change
            
            # Second approach: sometimes try substituting higher emission vehicles 
            # with lower emission ones
            if random.random() < 0.3:  # 30% chance to try substitution
                # Get vehicle types with non-zero allocation
                used_types = [vt for vt in solution.keys() if solution[vt] > 0]
                
                if used_types:
                    # Pick a random vehicle to reduce
                    reduce_type = random.choice(used_types)
                    
                    # Find its position in the sorted list
                    reduce_idx = next((i for i, v in enumerate(sorted_vehicles) 
                                     if v['vehicle_type'] == reduce_type), -1)
                    
                    # If we can find a better (lower emission) vehicle, try to substitute
                    if reduce_idx > 0:
                        mutation_happened = True
                        # Reduce this vehicle
                        mutated_solution[reduce_type] -= 1
                        
                        # Increase a better (lower emission) vehicle
                        better_vehicle = sorted_vehicles[random.randint(0, reduce_idx-1)]
                        mutated_solution[better_vehicle['vehicle_type']] += 1
            
            # If no mutation occurred, force at least one change
            if not mutation_happened:
                vehicle_type = random.choice(list(mutated_solution.keys()))
                if mutated_solution[vehicle_type] > 0:
                    mutated_solution[vehicle_type] -= 1
                else:
                    mutated_solution[vehicle_type] += 1
            
            # Check if valid
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            # Try to repair it if invalid
            repaired = self._repair_solution(mutated_solution, size_distance)
            if self.is_valid_solution(repaired, size_distance):
                return repaired
                
            attempts += 1
        
        # If all attempts fail, return original
        return solution

    def optimize(self, size_distance: Tuple, generations: int = 200) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        # Generate diverse initial population
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        no_improvement_count = 0
        
        # Track best solutions for comparison
        emissions_by_generation = []
        
        for gen in range(generations):
            # Calculate fitness for entire population
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Check if we have a new best solution
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
                no_improvement_count = 0
                
                # Calculate and save emissions for this generation
                best_emissions = self.calculate_emissions(best_solution, size_distance)
                emissions_by_generation.append(best_emissions)
            else:
                no_improvement_count += 1
            
            # Early stopping if no improvement for many generations
            if no_improvement_count >= 50:
                break
            
            # Select parents (use tournament selection)
            parents = []
            tournament_size = 3
            for _ in range(len(population) // 2):
                tournament = random.sample(fitness_scores, tournament_size)
                tournament.sort(key=lambda x: x[1], reverse=True)
                parents.append(tournament[0][0])
            
            # Create next generation
            next_generation = []
            
            # Elitism: keep top 10% solutions
            elite_count = max(1, len(population) // 10)
            next_generation.extend([score[0] for score in fitness_scores[:elite_count]])
            
            # Fill the rest through crossover and mutation
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                
                # Apply mutation with varying rates based on generation
                # Higher mutation early, lower mutation later
                mutation_rate = 0.3 * (1 - min(1.0, gen / (generations * 0.7)))
                
                child1 = self.mutate(child1, size_distance, mutation_rate)
                child2 = self.mutate(child2, size_distance, mutation_rate)
                
                next_generation.append(child1)
                if len(next_generation) < len(population):
                    next_generation.append(child2)
            
            population = next_generation
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            size_distance_emissions = 0
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    
                    # Calculate distance covered by this vehicle type
                    demand = vehicle_data['demand']
                    total_capacity = sum(best_solution[v['vehicle_type']] * v['yearly_range'] 
                                        for v in self.vehicles_by_size_distance[size_distance])
                    
                    # Calculate proportional demand for this vehicle type
                    vehicle_capacity = num_vehicles * vehicle_data['yearly_range']
                    vehicle_demand_share = (vehicle_capacity / total_capacity) * demand if total_capacity > 0 else 0
                    
                    # Calculate emissions
                    emissions = vehicle_data['carbon_emissions_per_km'] * vehicle_demand_share
                    size_distance_emissions += emissions
                    total_carbon_emissions += emissions

                    # Calculate evaluation metrics
                    utilization = self.calculate_utilization(
                        vehicle_data['demand'],
                        num_vehicles,
                        vehicle_data['yearly_range']
                    )
                    
                    demand_fulfillment = self.calculate_demand_fulfillment(
                        num_vehicles,
                        max_vehicles
                    )

                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2),
                        "Carbon Emissions": round(emissions, 2),
                        # "Emissions Per KM": vehicle_data['carbon_emissions_per_km']
                    })

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with pure carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "FINAL_CODES/topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    # results_df.to_csv("optimized_fleet_allocation_carbon.csv", index=False)


Running optimization with pure carbon emissions minimization...

Total Carbon Emissions: 2776694.85

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     BEV  Electricity               9             9   
1   Size S1, Distance D1     LNG          LNG               1             9   
2   Size S1, Distance D2     LNG          LNG              13            26   
3   Size S1, Distance D3     LNG          LNG              17            33   
4   Size S1, Distance D4     LNG          LNG               3             5   
5   Size S2, Distance D1     BEV  Electricity              10            10   
6   Size S2, Distance D1     LNG          LNG               1            10   
7   Size S2, Distance D2     LNG          LNG               7            14   
8   Size S2, Distance D3     LNG          LNG               4             8   
9   Size S2, Distance D4     LNG          LNG               1             2   
1

In [ ]:
import os
import pandas as pd

# Define the input and output folder paths
input_folder = "topsis_result/"
output_folder = "optimized_fleet_results_carbon_new/"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

def process_all_years():
    """Processes all available yearly data files and stores results."""
    for file in os.listdir(input_folder):
        if file.endswith(".csv"):
            year = file.split("_")[-1].split(".")[0]  # Extract year from filename
            csv_path = os.path.join(input_folder, file)
            
            print(f"\nProcessing year: {year}")
            results_df = main(csv_path)
            
            # Save results for the year
            output_file = os.path.join(output_folder, f"optimized_fleet_allocation_carbon_{year}.csv")
            results_df.to_csv(output_file, index=False)
            print(f"Results saved for {year} at: {output_file}")

    summary_df = pd.DataFrame(yearly_costs)
    summary_file = os.path.join(output_folder, "yearly_total_carbon_summary.csv")
    summary_df.to_csv(summary_file, index=False)
    print(f"\nYearly total cost summary saved to {summary_file}")

if __name__ == "__main__":
    process_all_years()



Processing year: 2023

Running optimization with pure carbon emissions minimization...

Total Carbon Emissions: 2696051.06

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     BEV  Electricity               9             9   
1   Size S1, Distance D2     LNG          LNG              13            26   
2   Size S1, Distance D3     LNG          LNG              17            33   
3   Size S1, Distance D4     LNG          LNG               3             5   
4   Size S2, Distance D1     BEV  Electricity              10            10   
5   Size S2, Distance D2     LNG          LNG               7            14   
6   Size S2, Distance D3     LNG          LNG               4             8   
7   Size S2, Distance D4     LNG          LNG               1             2   
8   Size S3, Distance D1     BEV  Electricity              30            30   
9   Size S3, Distance D2     LNG          LNG            

In [ ]:
# FINALLLLL DONEEEE

In [7]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'Allocation' : row['Allocation'],
                'Size': row['Size'],
                'Distance_demand': row['Distance_demand'],
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            # Add a small buffer to ensure demand is met even with rounding issues
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return min(100, (demand / num_vehicles) / yearly_range * 100)  # Cap at 100%

    def calculate_demand_fulfillment(self, total_capacity: float, demand: float) -> float:
        """Calculate demand fulfillment percentage"""
        if demand == 0:
            return 100.0
        return min(100.0, (total_capacity / demand) * 100)

    def demand_deficit(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate how much demand is not met by the current solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        demand = vehicles[0]['demand']
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        
        return max(0, demand - total_capacity)

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        # Calculate how much of the demand is satisfied by each vehicle type
        demand = vehicles[0]['demand']
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        
        # If demand is not fully met, return infinity (extremely high penalty)
        if total_capacity < demand:
            return float('inf')
        
        # Calculate actual distance covered by each vehicle type
        remaining_demand = demand
        
        # Sort vehicles by emissions (lowest first) to prioritize cleaner vehicles when allocating demand
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        for vehicle in sorted_vehicles:
            vehicle_type = vehicle['vehicle_type']
            num_vehicles = solution[vehicle_type]
            
            if num_vehicles > 0:
                # Calculate how much distance this vehicle type can cover
                vehicle_capacity = num_vehicles * vehicle['yearly_range']
                assigned_demand = min(vehicle_capacity, remaining_demand)
                remaining_demand -= assigned_demand
                
                # Calculate emissions for this portion of demand
                emissions = vehicle['carbon_emissions_per_km'] * assigned_demand
                total_emissions += emissions
                
                # If all demand is allocated, break
                if remaining_demand <= 0:
                    break
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (meets vehicle limits and demand)"""
        # Check if we don't exceed max vehicles per group
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Check if demand is fully met (strict requirement)
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        demand = vehicles[0]['demand']
        
        # Must satisfy at least 99.9% of demand to handle floating point imprecision
        return total_capacity >= (0.999 * demand)

    # def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
    #     """
    #     Calculate fitness score with primary focus on demand fulfillment,
    #     then carbon emissions, then efficiency
    #     """
    #     # First check: is the solution valid?
    #     if not self.is_valid_solution(solution, size_distance):
    #         # Calculate deficit to guide evolution towards valid solutions
    #         deficit = self.demand_deficit(solution, size_distance)
    #         # Negative score proportional to deficit - helps genetic algorithm converge better
    #         return -1000000 * deficit
        
    #     # Solution is valid, so calculate emissions
    #     emissions = self.calculate_emissions(solution, size_distance)
        
    #     # If emissions are infinite (demand not met), this shouldn't happen but handle it
    #     if emissions == float('inf'):
    #         return float('-inf')
        
    #     # Inverse relationship: lower emissions = higher fitness
    #     emissions_score = 1000000 / (emissions + 1)
        
    #     # Add a small component for efficiency (fewer vehicles is better)
    #     vehicles_count = sum(solution.values())
    #     max_vehicles = self.max_vehicles_by_group[size_distance]
    #     if vehicles_count > 0:
    #         efficiency_score = max_vehicles / vehicles_count
    #     else:
    #         efficiency_score = 0
        
    #     # Calculate diversity score (mix of vehicle types is sometimes better for robustness)
    #     vehicle_types_used = sum(1 for v in solution.values() if v > 0)
    #     diversity_factor = vehicle_types_used / len(solution) if len(solution) > 0 else 0
            
    #     # Return combined score with emphasis on emissions
    #     return emissions_score * 0.9 + efficiency_score * 0.07 + diversity_factor * 0.03


    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness score with primary focus on demand fulfillment,
        then carbon emissions, then efficiency, incorporating TOPSIS score.
        """
        if not self.is_valid_solution(solution, size_distance):
            deficit = self.demand_deficit(solution, size_distance)
            return -1000000 * deficit
        
        emissions = self.calculate_emissions(solution, size_distance)
        if emissions == float('inf'):
            return float('-inf')
        
        emissions_score = 1000000 / (emissions + 1)
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        weighted_topsis = 0
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        vehicles_count = sum(solution.values())
        max_vehicles = self.max_vehicles_by_group[size_distance]
        efficiency_score = max_vehicles / vehicles_count if vehicles_count > 0 else 0
        
        # vehicle_types_used = sum(1 for v in solution.values() if v > 0)
        # diversity_factor = vehicle_types_used / len(solution) if len(solution) > 0 else 0
        
        return emissions_score+ efficiency_score


    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate diverse initial population with guaranteed demand fulfillment"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        demand = vehicles[0]['demand']
        
        # Get all vehicle types
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        # Create solutions with diversity - but ensure all meet demand
        attempts = 0
        max_attempts = population_size * 10  # Limit attempts to avoid infinite loops
        
        # Include greedy solutions in the initial population for quick convergence
        # 1. Sort vehicles by emissions (low to high)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        greedy_solution = {v_type: 0 for v_type in vehicle_types}
        
        remaining_demand = demand
        for vehicle in sorted_vehicles:
            vehicle_type = vehicle['vehicle_type']
            while remaining_demand > 0 and sum(greedy_solution.values()) < max_vehicles:
                greedy_solution[vehicle_type] += 1
                remaining_demand -= vehicle['yearly_range']
        
        if self.is_valid_solution(greedy_solution, size_distance):
            population.append(greedy_solution)
        
        # 2. Sort by range (high to low) for efficiency
        sorted_by_range = sorted(vehicles, key=lambda v: v['yearly_range'], reverse=True)
        range_solution = {v_type: 0 for v_type in vehicle_types}
        
        remaining_demand = demand
        for vehicle in sorted_by_range:
            vehicle_type = vehicle['vehicle_type']
            while remaining_demand > 0 and sum(range_solution.values()) < max_vehicles:
                range_solution[vehicle_type] += 1
                remaining_demand -= vehicle['yearly_range']
        
        if self.is_valid_solution(range_solution, size_distance):
            population.append(range_solution)
        
        # Generate more random solutions
        while len(population) < population_size and attempts < max_attempts:
            attempts += 1
            
            # Create a new random solution
            solution = {v_type: 0 for v_type in vehicle_types}
            
            # Randomly distribute vehicles to meet demand
            remaining_demand = demand
            while remaining_demand > 0 and sum(solution.values()) < max_vehicles:
                # Select a random vehicle type
                vehicle_type = random.choice(vehicle_types)
                
                # Add one of this type
                solution[vehicle_type] += 1
                remaining_demand -= vehicle_ranges[vehicle_type]
            
            # Check if valid and not duplicate
            if self.is_valid_solution(solution, size_distance) and solution not in population:
                population.append(solution)
        
        # If we couldn't generate enough solutions, make duplicates with small mutations
        while len(population) < population_size:
            if not population:  # Safety check
                base_solution = {v_type: 0 for v_type in vehicle_types}
                # Add vehicles until demand is met
                remaining_demand = demand
                for vt in vehicle_types:
                    while remaining_demand > 0:
                        base_solution[vt] += 1
                        remaining_demand -= vehicle_ranges[vt]
                    if remaining_demand <= 0:
                        break
                population.append(base_solution)
            
            # Take a random solution and mutate it
            base = random.choice(population)
            mutated = self.mutate(base, size_distance, 0.3)
            
            if self.is_valid_solution(mutated, size_distance):
                population.append(mutated)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions with guaranteed demand fulfillment"""
        attempts = 0
        max_attempts = 20
        vehicles = self.vehicles_by_size_distance[size_distance]
        demand = vehicles[0]['demand']
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        while attempts < max_attempts:
            # Use two-point crossover for better diversity
            vehicle_types = list(parent1.keys())
            
            if len(vehicle_types) < 3:
                # For small number of types, use single point crossover
                crossover_point = random.randint(1, len(vehicle_types) - 1)
                
                child1 = {}
                child2 = {}
                
                for i, vehicle_type in enumerate(vehicle_types):
                    if i < crossover_point:
                        child1[vehicle_type] = parent1[vehicle_type]
                        child2[vehicle_type] = parent2[vehicle_type]
                    else:
                        child1[vehicle_type] = parent2[vehicle_type]
                        child2[vehicle_type] = parent1[vehicle_type]
            else:
                # For more types, use two-point crossover
                points = sorted(random.sample(range(1, len(vehicle_types)), 2))
                
                child1 = {}
                child2 = {}
                
                for i, vehicle_type in enumerate(vehicle_types):
                    if i < points[0] or i >= points[1]:
                        child1[vehicle_type] = parent1[vehicle_type]
                        child2[vehicle_type] = parent2[vehicle_type]
                    else:
                        child1[vehicle_type] = parent2[vehicle_type]
                        child2[vehicle_type] = parent1[vehicle_type]
            
            # Ensure demand is met for both children
            child1_capacity = sum(child1[vt] * vehicle_ranges[vt] for vt in vehicle_types)
            child2_capacity = sum(child2[vt] * vehicle_ranges[vt] for vt in vehicle_types)
            
            # If both meet demand and are valid, return them
            valid1 = self.is_valid_solution(child1, size_distance)
            valid2 = self.is_valid_solution(child2, size_distance)
            
            if valid1 and valid2:
                return child1, child2
            
            # If only one is valid, repair the other
            if valid1 and not valid2:
                child2 = self._repair_solution(child2, size_distance)
                if self.is_valid_solution(child2, size_distance):
                    return child1, child2
            elif valid2 and not valid1:
                child1 = self._repair_solution(child1, size_distance)
                if self.is_valid_solution(child1, size_distance):
                    return child1, child2
            else:
                # Both invalid, repair both
                child1 = self._repair_solution(child1, size_distance)
                child2 = self._repair_solution(child2, size_distance)
                if self.is_valid_solution(child1, size_distance) and self.is_valid_solution(child2, size_distance):
                    return child1, child2
                    
            attempts += 1
        
        # If failed to create valid children, use the parents if they're valid
        if self.is_valid_solution(parent1, size_distance) and self.is_valid_solution(parent2, size_distance):
            return parent1.copy(), parent2.copy()
        
        # If one parent is invalid, return two copies of the valid one
        if self.is_valid_solution(parent1, size_distance):
            return parent1.copy(), parent1.copy()
        if self.is_valid_solution(parent2, size_distance):
            return parent2.copy(), parent2.copy()
        
        # If both parents are invalid (shouldn't happen), create a basic valid solution
        basic_solution = self._create_basic_valid_solution(size_distance)
        return basic_solution, basic_solution
    
    def _create_basic_valid_solution(self, size_distance: Tuple) -> Dict:
        """Create a simple valid solution that meets demand"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        demand = vehicles[0]['demand']
        
        # Sort vehicles by emissions (low to high)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        solution = {v_type: 0 for v_type in vehicle_types}
        remaining_demand = demand
        
        # Add vehicles until demand is met
        for vehicle in sorted_vehicles:
            vehicle_type = vehicle['vehicle_type']
            while remaining_demand > 0:
                solution[vehicle_type] += 1
                remaining_demand -= vehicle['yearly_range']
                if remaining_demand <= 0:
                    break
            if remaining_demand <= 0:
                break
        
        return solution
    
    def _repair_solution(self, solution: Dict, size_distance: Tuple) -> Dict:
        """Repair an invalid solution to ensure it meets demand requirements"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        demand = vehicles[0]['demand']
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        # Make a copy to avoid modifying the original
        repaired = solution.copy()
        
        # Calculate current capacity
        total_capacity = sum(repaired[vt] * vehicle_ranges[vt] for vt in vehicle_types)
        total_vehicles = sum(repaired.values())
        
        # If we exceed max vehicles, reduce until within limits
        if total_vehicles > max_vehicles:
            # Sort vehicle types by efficiency (range per vehicle)
            efficiency_ranking = sorted(vehicle_types, 
                                      key=lambda vt: vehicle_ranges[vt], 
                                      reverse=True)
            
            # Reduce vehicles starting with least efficient types
            for vt in reversed(efficiency_ranking):
                while total_vehicles > max_vehicles and repaired[vt] > 0:
                    repaired[vt] -= 1
                    total_vehicles -= 1
                    total_capacity -= vehicle_ranges[vt]
        
        # If demand is not met, add vehicles until it is
        if total_capacity < demand:
            # Sort by emissions efficiency (lower emissions first)
            emissions_ranking = sorted(vehicles, 
                                     key=lambda v: v['carbon_emissions_per_km'])
            
            # First try adding lowest emission vehicles
            for vehicle in emissions_ranking:
                vt = vehicle['vehicle_type']
                while total_capacity < demand and total_vehicles < max_vehicles:
                    repaired[vt] += 1
                    total_vehicles += 1
                    total_capacity += vehicle_ranges[vt]
                
                if total_capacity >= demand:
                    break
            
            # If still not meeting demand and at max vehicles, swap less efficient vehicles
            # for more efficient ones
            if total_capacity < demand and total_vehicles >= max_vehicles:
                # Sort by range (highest first)
                range_ranking = sorted(vehicle_types, 
                                     key=lambda vt: vehicle_ranges[vt], 
                                     reverse=True)
                
                # Try replacing less efficient vehicles with more efficient ones
                for high_range_vt in range_ranking:
                    for low_range_vt in reversed(range_ranking):
                        if high_range_vt == low_range_vt:
                            continue
                        
                        # Only swap if it would increase capacity
                        if vehicle_ranges[high_range_vt] > vehicle_ranges[low_range_vt] and repaired[low_range_vt] > 0:
                            repaired[low_range_vt] -= 1
                            repaired[high_range_vt] += 1
                            
                            # Recalculate capacity
                            total_capacity = sum(repaired[vt] * vehicle_ranges[vt] for vt in vehicle_types)
                            
                            if total_capacity >= demand:
                                break
                    
                    if total_capacity >= demand:
                        break
        
        return repaired

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.2) -> Dict:
        """Mutate a solution with focus on emissions reduction while maintaining demand fulfillment"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        # Sort vehicles by carbon emissions (lowest first)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        # Try multiple mutation strategies
        attempts = 0
        max_attempts = 20
        
        while attempts < max_attempts:
            attempts += 1
            mutated = solution.copy()
            mutation_happened = False
            
            # Strategy 1: Random adjustment - increase or decrease random vehicles
            for vt in vehicle_types:
                if random.random() < mutation_rate:
                    mutation_happened = True
                    change = random.choice([-1, 1])
                    if change == -1 and mutated[vt] > 0:
                        mutated[vt] -= 1
                    elif change == 1:
                        mutated[vt] += 1
            
            # Strategy 2: Emission reduction - swap high emission for low emission
            if random.random() < 0.4:  # 40% chance
                # Find a vehicle with non-zero allocation
                used_types = [vt for vt in vehicle_types if mutated[vt] > 0]
                if used_types:
                    # Sort by emissions
                    used_types_sorted = sorted(used_types, 
                                             key=lambda vt: next(v['carbon_emissions_per_km'] for v in vehicles if v['vehicle_type'] == vt))
                    
                    # Try to reduce a high-emission vehicle
                    if len(used_types_sorted) > 1:
                        high_emission_vt = used_types_sorted[-1]
                        low_emission_vt = used_types_sorted[0]
                        
                        # Only if reducing doesn't make solution invalid
                        temp = mutated.copy()
                        temp[high_emission_vt] -= 1
                        temp[low_emission_vt] += 1
                        
                        if self.is_valid_solution(temp, size_distance):
                            mutated = temp
                            mutation_happened = True
            
            # Strategy 3: Vehicle reduction - if possible, reduce total vehicles
            if random.random() < 0.3:  # 30% chance
                # Sort by range (highest first)
                range_ranking = sorted(vehicle_types, 
                                     key=lambda vt: vehicle_ranges[vt], 
                                     reverse=True)
                
                # Try to replace two low-range vehicles with one high-range vehicle
                for high_range_vt in range_ranking:
                    for low_range_vt in reversed(range_ranking):
                        if high_range_vt == low_range_vt or mutated[low_range_vt] < 2:
                            continue
                        
                        temp = mutated.copy()
                        temp[low_range_vt] -= 2
                        temp[high_range_vt] += 1
                        
                        if self.is_valid_solution(temp, size_distance):
                            mutated = temp
                            mutation_happened = True
                            break
                    
                    if mutation_happened:
                        break
            
            # If no mutation happened, force a simple change
            if not mutation_happened:
                vt = random.choice(vehicle_types)
                temp = mutated.copy()
                if random.random() < 0.5 and temp[vt] > 0:
                    temp[vt] -= 1
                else:
                    temp[vt] += 1
                
                if self.is_valid_solution(temp, size_distance):
                    mutated = temp
                    mutation_happened = True
            
            # Check if valid
            if self.is_valid_solution(mutated, size_distance):
                return mutated
            
            # Try to repair if invalid
            repaired = self._repair_solution(mutated, size_distance)
            if self.is_valid_solution(repaired, size_distance):
                return repaired
        
        # If all attempts fail, return original solution or a repaired version of it
        repaired_original = self._repair_solution(solution, size_distance)
        if self.is_valid_solution(repaired_original, size_distance):
            return repaired_original
        
        # Last resort: create a basic valid solution
        return self._create_basic_valid_solution(size_distance)

    def optimize(self, size_distance: Tuple, generations: int = 250) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        # Generate diverse initial population
        population_size = 150
        population = self.generate_initial_population(size_distance, population_size)
        
        best_solution = None
        best_fitness = float('-inf')
        no_improvement_count = 0
        
        # Track best solutions for comparison
        emissions_by_generation = []
        
        for gen in range(generations):
            # Calculate fitness for entire population
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Check if we have a new best solution
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
                no_improvement_count = 0
                
                # Calculate and save emissions for this generation
                best_emissions = self.calculate_emissions(best_solution, size_distance)
                emissions_by_generation.append(best_emissions)
            else:
                no_improvement_count += 1
            
            # Early stopping if no improvement for many generations
            if no_improvement_count >= 50:
                break
            
            # Select parents (use tournament selection)
            parents = []
            tournament_size = max(3, len(population) // 20)  # Dynamic tournament size
            for _ in range(len(population)):
                tournament = random.sample(fitness_scores, tournament_size)
                tournament.sort(key=lambda x: x[1], reverse=True)
                parents.append(tournament[0][0])
            
            # Create next generation
            next_generation = []
            
            # Elitism: keep top solutions
            elite_count = max(2, len(population) // 10)
            next_generation.extend([score[0] for score in fitness_scores[:elite_count]])
            
            # Fill the rest through crossover and mutation
            while len(next_generation) < population_size:
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                
                # Apply mutation with varying rates based on generation
                # Higher mutation early, lower mutation later
                mutation_rate = 0.3 * (1 - min(1.0, gen / (generations * 0.7)))
                
                child1 = self.mutate(child1, size_distance, mutation_rate)
                child2 = self.mutate(child2, size_distance, mutation_rate)
                
                next_generation.append(child1)
                if len(next_generation) < population_size:
                    next_generation.append(child2)
            
            population = next_generation
        
        # Validate that best solution meets demand
        if best_solution and not self.is_valid_solution(best_solution, size_distance):
            best_solution = self._repair_solution(best_solution, size_distance)
            
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            print(f"Optimizing for Size: {size_distance[0]}, Distance: {size_distance[1]}...")
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            # Validate solution meets demand
            vehicles = self.vehicles_by_size_distance[size_distance] 
            demand = vehicles[0]['demand']
            total_capacity = sum(best_solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
            
            if total_capacity < demand:
                print(f"Warning: Solution does not meet demand for {size_distance}. Repairing...")
                best_solution = self._repair_solution(best_solution, size_distance)
                total_capacity = sum(best_solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
            
            demand_fulfillment = self.calculate_demand_fulfillment(total_capacity, demand)
            size_distance_emissions = 0
            
            # Sort vehicles by emissions to allocate demand to lowest emission vehicles first
            sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
            remaining_demand = demand
            
            # Calculate emissions by allocating demand optimally
            for vehicle in sorted_vehicles:
                vehicle_type = vehicle['vehicle_type']
                num_vehicles = best_solution[vehicle_type]
                
                if num_vehicles > 0:
                    # Calculate how much this vehicle type can cover
                    vehicle_capacity = num_vehicles * vehicle['yearly_range']
                    assigned_demand = min(vehicle_capacity, remaining_demand)
                    remaining_demand -= assigned_demand
                    
                    # If no more demand to assign, vehicle is unused
                    if assigned_demand <= 0:
                        continue
                    
                    # Calculate emissions
                    emissions = vehicle['carbon_emissions_per_km'] * assigned_demand
                    size_distance_emissions += emissions
                    
                    # Calculate utilization for this vehicle type
                    utilization = self.calculate_utilization(
                        assigned_demand,
                        num_vehicles,
                        vehicle['yearly_range']
                    )

                    Allocation = vehicle.get('Allocation')
                    Size = vehicle.get('Size')
                    Distance_demand = vehicle.get('Distance_demand')
                    
                    results.append({
                        # "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        'Allocation' : Allocation,
                        'Size': Size,
                        'Distance_demand': Distance_demand,
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": demand,
                        "Assigned Demand": round(assigned_demand, 2),
                        "Yearly Range": vehicle['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2),
                        "Carbon Emissions": round(emissions, 2),
                        "Emissions Per KM": round(vehicle['carbon_emissions_per_km'], 4)
                    })
            
            total_carbon_emissions += size_distance_emissions
            
            # Verify all demand is allocated
            if abs(remaining_demand) > 0.01:  # Allow small floating-point error
                print(f"Warning: {remaining_demand:.2f} km of demand not allocated for {size_distance}")

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with pure carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "FINAL_CODES/topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)


Running optimization with pure carbon emissions minimization...
Optimizing for Size: S1, Distance: D1...
Optimizing for Size: S1, Distance: D2...
Optimizing for Size: S1, Distance: D3...
Optimizing for Size: S1, Distance: D4...
Optimizing for Size: S2, Distance: D1...
Optimizing for Size: S2, Distance: D2...
Optimizing for Size: S2, Distance: D3...
Optimizing for Size: S2, Distance: D4...
Optimizing for Size: S3, Distance: D1...
Optimizing for Size: S3, Distance: D2...
Optimizing for Size: S3, Distance: D3...
Optimizing for Size: S3, Distance: D4...
Optimizing for Size: S4, Distance: D1...
Optimizing for Size: S4, Distance: D2...
Optimizing for Size: S4, Distance: D3...
Optimizing for Size: S4, Distance: D4...

Total Carbon Emissions: 821042.14

Optimized Fleet Allocation:
     Allocation Size Distance_demand Vehicle         Fuel  no_of_vehicles  \
0   BEV_S1_2023   S1              D1     BEV  Electricity               9   
1   LNG_S1_2023   S1              D2     LNG       BioLNG    

In [9]:
# import os
# import pandas as pd

# # Define the input and output folder paths
input_folder = "topsis_result/"
output_folder = "data/optimized_fleet_results_carbon/"

# # Ensure the output folder exists
# os.makedirs(output_folder, exist_ok=True)

# def process_all_years():
#     """Processes all available yearly data files and stores results."""
#     for file in os.listdir(input_folder):
#         if file.endswith(".csv"):
#             year = file.split("_")[-1].split(".")[0]  # Extract year from filename
#             csv_path = os.path.join(input_folder, file)
            
#             print(f"\nProcessing year: {year}")
#             results_df = main(csv_path)
            
#             # Save results for the year
#             output_file = os.path.join(output_folder, f"optimized_fleet_allocation_carbon_{year}.csv")
#             results_df.to_csv(output_file, index=False)
#             print(f"Results saved for {year} at: {output_file}")

#     summary_df = pd.DataFrame(total_carbon_emissions)
#     summary_file = os.path.join(output_folder, "yearly_total_carbon_summary.csv")
#     summary_df.to_csv(summary_file, index=False)
#     print(f"\nYearly total cost summary saved to {summary_file}")




# import os
# import pandas as pd

# input_folder = "path/to/input"
# output_folder = "path/to/output"

def process_all_years():
    """Processes all available yearly data files and stores results."""
    yearly_emissions = []  # List to store total emissions per year

    for file in os.listdir(input_folder):
        if file.endswith(".csv"):
            year = file.split("_")[-1].split(".")[0]  # Extract year from filename
            csv_path = os.path.join(input_folder, file)

            print(f"\nProcessing year: {year}")
            results_df = main(csv_path)  # Assume main() processes and returns a DataFrame

            # Save results for the year
            output_file = os.path.join(output_folder, f"optimized_fleet_allocation_carbon_{year}.csv")
            results_df.to_csv(output_file, index=False)
            print(f"Results saved for {year} at: {output_file}")

            # Extract total carbon emissions
            if "Total Carbon Emissions" in results_df.columns:
                total_emissions = results_df["Total Carbon Emissions"].sum()
                yearly_emissions.append({"Year": year, "Total Carbon Emissions": total_emissions})
            else:
                print(f"Warning: 'Total Carbon Emissions' column not found in {file}")

    # Save yearly emissions summary
    emissions_df = pd.DataFrame(yearly_emissions)
    emissions_summary_file = os.path.join(output_folder, "yearly_total_carbon_emissions.csv")
    emissions_df.to_csv(emissions_summary_file, index=False)
    print(f"\nYearly total carbon emissions saved at: {emissions_summary_file}")


if __name__ == "__main__":
    process_all_years()



Processing year: 2023

Running optimization with pure carbon emissions minimization...
Optimizing for Size: S1, Distance: D1...
Optimizing for Size: S1, Distance: D2...
Optimizing for Size: S1, Distance: D3...
Optimizing for Size: S1, Distance: D4...
Optimizing for Size: S2, Distance: D1...
Optimizing for Size: S2, Distance: D2...
Optimizing for Size: S2, Distance: D3...
Optimizing for Size: S2, Distance: D4...
Optimizing for Size: S3, Distance: D1...
Optimizing for Size: S3, Distance: D2...
Optimizing for Size: S3, Distance: D3...
Optimizing for Size: S3, Distance: D4...
Optimizing for Size: S4, Distance: D1...
Optimizing for Size: S4, Distance: D2...
Optimizing for Size: S4, Distance: D3...
Optimizing for Size: S4, Distance: D4...

Total Carbon Emissions: 821042.14

Optimized Fleet Allocation:
              Allocation Vehicle         Fuel  no_of_vehicles  Max Vehicles  \
0   Size S1, Distance D1     BEV  Electricity               9             9   
1   Size S1, Distance D2     LNG  

In [ ]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'carbon emissions per km': 'carbon_emissions_per_km',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.total_carbon_emissions = 0
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand'],
                'fuel': row['Fuel']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            # Add a small buffer to ensure demand is met even with rounding issues
            max_vehicles[key] = math.ceil(demand / max_yearly_range * 1.05)
        return max_vehicles

    def calculate_utilization(self, demand: float, num_vehicles: int, yearly_range: float) -> float:
        """Calculate utilization metric for a vehicle type"""
        if num_vehicles == 0 or yearly_range == 0:
            return 0
        return min(100, (demand / num_vehicles) / yearly_range * 100)  # Cap at 100%

    def calculate_demand_fulfillment(self, total_capacity: float, demand: float) -> float:
        """Calculate demand fulfillment percentage"""
        if demand == 0:
            return 100.0
        return min(100.0, (total_capacity / demand) * 100)

    def demand_deficit(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate how much demand is not met by the current solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        demand = vehicles[0]['demand']
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        
        return max(0, demand - total_capacity)

    def calculate_emissions(self, solution: Dict, size_distance: Tuple) -> float:
        """Calculate total carbon emissions for a solution"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        # Calculate how much of the demand is satisfied by each vehicle type
        demand = vehicles[0]['demand']
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        
        # If demand is not fully met, return infinity (extremely high penalty)
        if total_capacity < demand:
            return float('inf')
        
        # Calculate actual distance covered by each vehicle type
        remaining_demand = demand
        
        # Sort vehicles by emissions (lowest first) to prioritize cleaner vehicles when allocating demand
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        for vehicle in sorted_vehicles:
            vehicle_type = vehicle['vehicle_type']
            num_vehicles = solution[vehicle_type]
            
            if num_vehicles > 0:
                # Calculate how much distance this vehicle type can cover
                vehicle_capacity = num_vehicles * vehicle['yearly_range']
                assigned_demand = min(vehicle_capacity, remaining_demand)
                remaining_demand -= assigned_demand
                
                # Calculate emissions for this portion of demand
                emissions = vehicle['carbon_emissions_per_km'] * assigned_demand
                total_emissions += emissions
                
                # If all demand is allocated, break
                if remaining_demand <= 0:
                    break
        
        return total_emissions

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the solution is valid (meets vehicle limits and demand)"""
        # Check if we don't exceed max vehicles per group
        if sum(solution.values()) > self.max_vehicles_by_group[size_distance]:
            return False
            
        # Check if demand is fully met (strict requirement)
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_capacity = sum(solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
        demand = vehicles[0]['demand']
        
        # Must satisfy at least 99.9% of demand to handle floating point imprecision
        return total_capacity >= (0.999 * demand)

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """
        Calculate fitness score with primary focus on demand fulfillment,
        then carbon emissions, then efficiency
        """
        # First check: is the solution valid?
        if not self.is_valid_solution(solution, size_distance):
            # Calculate deficit to guide evolution towards valid solutions
            deficit = self.demand_deficit(solution, size_distance)
            # Negative score proportional to deficit - helps genetic algorithm converge better
            return -1000000 * deficit
        
        # Solution is valid, so calculate emissions
        emissions = self.calculate_emissions(solution, size_distance)
        
        # If emissions are infinite (demand not met), this shouldn't happen but handle it
        if emissions == float('inf'):
            return float('-inf')
        
        # Inverse relationship: lower emissions = higher fitness
        emissions_score = 1000000 / (emissions + 1)
        
        # Add a small component for efficiency (fewer vehicles is better)
        vehicles_count = sum(solution.values())
        max_vehicles = self.max_vehicles_by_group[size_distance]
        if vehicles_count > 0:
            efficiency_score = max_vehicles / vehicles_count
        else:
            efficiency_score = 0
        
        # Calculate diversity score (mix of vehicle types is sometimes better for robustness)
        vehicle_types_used = sum(1 for v in solution.values() if v > 0)
        diversity_factor = vehicle_types_used / len(solution) if len(solution) > 0 else 0
            
        # Return combined score with emphasis on emissions
        return emissions_score * 0.9 + efficiency_score * 0.07 + diversity_factor * 0.03

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 150) -> List[Dict]:
        """Generate diverse initial population with guaranteed demand fulfillment"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        demand = vehicles[0]['demand']
        
        # Get all vehicle types
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        # Create solutions with diversity - but ensure all meet demand
        attempts = 0
        max_attempts = population_size * 10  # Limit attempts to avoid infinite loops
        
        # Include greedy solutions in the initial population for quick convergence
        # 1. Sort vehicles by emissions (low to high)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        greedy_solution = {v_type: 0 for v_type in vehicle_types}
        
        remaining_demand = demand
        for vehicle in sorted_vehicles:
            vehicle_type = vehicle['vehicle_type']
            while remaining_demand > 0 and sum(greedy_solution.values()) < max_vehicles:
                greedy_solution[vehicle_type] += 1
                remaining_demand -= vehicle['yearly_range']
        
        if self.is_valid_solution(greedy_solution, size_distance):
            population.append(greedy_solution)
        
        # 2. Sort by range (high to low) for efficiency
        sorted_by_range = sorted(vehicles, key=lambda v: v['yearly_range'], reverse=True)
        range_solution = {v_type: 0 for v_type in vehicle_types}
        
        remaining_demand = demand
        for vehicle in sorted_by_range:
            vehicle_type = vehicle['vehicle_type']
            while remaining_demand > 0 and sum(range_solution.values()) < max_vehicles:
                range_solution[vehicle_type] += 1
                remaining_demand -= vehicle['yearly_range']
        
        if self.is_valid_solution(range_solution, size_distance):
            population.append(range_solution)
        
        # Generate more random solutions
        while len(population) < population_size and attempts < max_attempts:
            attempts += 1
            
            # Create a new random solution
            solution = {v_type: 0 for v_type in vehicle_types}
            
            # Randomly distribute vehicles to meet demand
            remaining_demand = demand
            while remaining_demand > 0 and sum(solution.values()) < max_vehicles:
                # Select a random vehicle type
                vehicle_type = random.choice(vehicle_types)
                
                # Add one of this type
                solution[vehicle_type] += 1
                remaining_demand -= vehicle_ranges[vehicle_type]
            
            # Check if valid and not duplicate
            if self.is_valid_solution(solution, size_distance) and solution not in population:
                population.append(solution)
        
        # If we couldn't generate enough solutions, make duplicates with small mutations
        while len(population) < population_size:
            if not population:  # Safety check
                base_solution = {v_type: 0 for v_type in vehicle_types}
                # Add vehicles until demand is met
                remaining_demand = demand
                for vt in vehicle_types:
                    while remaining_demand > 0:
                        base_solution[vt] += 1
                        remaining_demand -= vehicle_ranges[vt]
                    if remaining_demand <= 0:
                        break
                population.append(base_solution)
            
            # Take a random solution and mutate it
            base = random.choice(population)
            mutated = self.mutate(base, size_distance, 0.3)
            
            if self.is_valid_solution(mutated, size_distance):
                population.append(mutated)
        
        return population

    def crossover(self, parent1: Dict, parent2: Dict, size_distance: Tuple) -> Tuple[Dict, Dict]:
        """Perform crossover between two parent solutions with guaranteed demand fulfillment"""
        attempts = 0
        max_attempts = 20
        vehicles = self.vehicles_by_size_distance[size_distance]
        demand = vehicles[0]['demand']
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        while attempts < max_attempts:
            # Use two-point crossover for better diversity
            vehicle_types = list(parent1.keys())
            
            if len(vehicle_types) < 3:
                # For small number of types, use single point crossover
                crossover_point = random.randint(1, len(vehicle_types) - 1)
                
                child1 = {}
                child2 = {}
                
                for i, vehicle_type in enumerate(vehicle_types):
                    if i < crossover_point:
                        child1[vehicle_type] = parent1[vehicle_type]
                        child2[vehicle_type] = parent2[vehicle_type]
                    else:
                        child1[vehicle_type] = parent2[vehicle_type]
                        child2[vehicle_type] = parent1[vehicle_type]
            else:
                # For more types, use two-point crossover
                points = sorted(random.sample(range(1, len(vehicle_types)), 2))
                
                child1 = {}
                child2 = {}
                
                for i, vehicle_type in enumerate(vehicle_types):
                    if i < points[0] or i >= points[1]:
                        child1[vehicle_type] = parent1[vehicle_type]
                        child2[vehicle_type] = parent2[vehicle_type]
                    else:
                        child1[vehicle_type] = parent2[vehicle_type]
                        child2[vehicle_type] = parent1[vehicle_type]
            
            # Ensure demand is met for both children
            child1_capacity = sum(child1[vt] * vehicle_ranges[vt] for vt in vehicle_types)
            child2_capacity = sum(child2[vt] * vehicle_ranges[vt] for vt in vehicle_types)
            
            # If both meet demand and are valid, return them
            valid1 = self.is_valid_solution(child1, size_distance)
            valid2 = self.is_valid_solution(child2, size_distance)
            
            if valid1 and valid2:
                return child1, child2
            
            # If only one is valid, repair the other
            if valid1 and not valid2:
                child2 = self._repair_solution(child2, size_distance)
                if self.is_valid_solution(child2, size_distance):
                    return child1, child2
            elif valid2 and not valid1:
                child1 = self._repair_solution(child1, size_distance)
                if self.is_valid_solution(child1, size_distance):
                    return child1, child2
            else:
                # Both invalid, repair both
                child1 = self._repair_solution(child1, size_distance)
                child2 = self._repair_solution(child2, size_distance)
                if self.is_valid_solution(child1, size_distance) and self.is_valid_solution(child2, size_distance):
                    return child1, child2
                    
            attempts += 1
        
        # If failed to create valid children, use the parents if they're valid
        if self.is_valid_solution(parent1, size_distance) and self.is_valid_solution(parent2, size_distance):
            return parent1.copy(), parent2.copy()
        
        # If one parent is invalid, return two copies of the valid one
        if self.is_valid_solution(parent1, size_distance):
            return parent1.copy(), parent1.copy()
        if self.is_valid_solution(parent2, size_distance):
            return parent2.copy(), parent2.copy()
        
        # If both parents are invalid (shouldn't happen), create a basic valid solution
        basic_solution = self._create_basic_valid_solution(size_distance)
        return basic_solution, basic_solution
    
    def _create_basic_valid_solution(self, size_distance: Tuple) -> Dict:
        """Create a simple valid solution that meets demand"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        demand = vehicles[0]['demand']
        
        # Sort vehicles by emissions (low to high)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        solution = {v_type: 0 for v_type in vehicle_types}
        remaining_demand = demand
        
        # Add vehicles until demand is met
        for vehicle in sorted_vehicles:
            vehicle_type = vehicle['vehicle_type']
            while remaining_demand > 0:
                solution[vehicle_type] += 1
                remaining_demand -= vehicle['yearly_range']
                if remaining_demand <= 0:
                    break
            if remaining_demand <= 0:
                break
        
        return solution
    
    def _repair_solution(self, solution: Dict, size_distance: Tuple) -> Dict:
        """Repair an invalid solution to ensure it meets demand requirements"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        demand = vehicles[0]['demand']
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        # Make a copy to avoid modifying the original
        repaired = solution.copy()
        
        # Calculate current capacity
        total_capacity = sum(repaired[vt] * vehicle_ranges[vt] for vt in vehicle_types)
        total_vehicles = sum(repaired.values())
        
        # If we exceed max vehicles, reduce until within limits
        if total_vehicles > max_vehicles:
            # Sort vehicle types by efficiency (range per vehicle)
            efficiency_ranking = sorted(vehicle_types, 
                                      key=lambda vt: vehicle_ranges[vt], 
                                      reverse=True)
            
            # Reduce vehicles starting with least efficient types
            for vt in reversed(efficiency_ranking):
                while total_vehicles > max_vehicles and repaired[vt] > 0:
                    repaired[vt] -= 1
                    total_vehicles -= 1
                    total_capacity -= vehicle_ranges[vt]
        
        # If demand is not met, add vehicles until it is
        if total_capacity < demand:
            # Sort by emissions efficiency (lower emissions first)
            emissions_ranking = sorted(vehicles, 
                                     key=lambda v: v['carbon_emissions_per_km'])
            
            # First try adding lowest emission vehicles
            for vehicle in emissions_ranking:
                vt = vehicle['vehicle_type']
                while total_capacity < demand and total_vehicles < max_vehicles:
                    repaired[vt] += 1
                    total_vehicles += 1
                    total_capacity += vehicle_ranges[vt]
                
                if total_capacity >= demand:
                    break
            
            # If still not meeting demand and at max vehicles, swap less efficient vehicles
            # for more efficient ones
            if total_capacity < demand and total_vehicles >= max_vehicles:
                # Sort by range (highest first)
                range_ranking = sorted(vehicle_types, 
                                     key=lambda vt: vehicle_ranges[vt], 
                                     reverse=True)
                
                # Try replacing less efficient vehicles with more efficient ones
                for high_range_vt in range_ranking:
                    for low_range_vt in reversed(range_ranking):
                        if high_range_vt == low_range_vt:
                            continue
                        
                        # Only swap if it would increase capacity
                        if vehicle_ranges[high_range_vt] > vehicle_ranges[low_range_vt] and repaired[low_range_vt] > 0:
                            repaired[low_range_vt] -= 1
                            repaired[high_range_vt] += 1
                            
                            # Recalculate capacity
                            total_capacity = sum(repaired[vt] * vehicle_ranges[vt] for vt in vehicle_types)
                            
                            if total_capacity >= demand:
                                break
                    
                    if total_capacity >= demand:
                        break
        
        return repaired

    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.2) -> Dict:
        """Mutate a solution with focus on emissions reduction while maintaining demand fulfillment"""
        vehicles = self.vehicles_by_size_distance[size_distance]
        vehicle_types = [v['vehicle_type'] for v in vehicles]
        vehicle_ranges = {v['vehicle_type']: v['yearly_range'] for v in vehicles}
        
        # Sort vehicles by carbon emissions (lowest first)
        sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        # Try multiple mutation strategies
        attempts = 0
        max_attempts = 20
        
        while attempts < max_attempts:
            attempts += 1
            mutated = solution.copy()
            mutation_happened = False
            
            # Strategy 1: Random adjustment - increase or decrease random vehicles
            for vt in vehicle_types:
                if random.random() < mutation_rate:
                    mutation_happened = True
                    change = random.choice([-1, 1])
                    if change == -1 and mutated[vt] > 0:
                        mutated[vt] -= 1
                    elif change == 1:
                        mutated[vt] += 1
            
            # Strategy 2: Emission reduction - swap high emission for low emission
            if random.random() < 0.4:  # 40% chance
                # Find a vehicle with non-zero allocation
                used_types = [vt for vt in vehicle_types if mutated[vt] > 0]
                if used_types:
                    # Sort by emissions
                    used_types_sorted = sorted(used_types, 
                                             key=lambda vt: next(v['carbon_emissions_per_km'] for v in vehicles if v['vehicle_type'] == vt))
                    
                    # Try to reduce a high-emission vehicle
                    if len(used_types_sorted) > 1:
                        high_emission_vt = used_types_sorted[-1]
                        low_emission_vt = used_types_sorted[0]
                        
                        # Only if reducing doesn't make solution invalid
                        temp = mutated.copy()
                        temp[high_emission_vt] -= 1
                        temp[low_emission_vt] += 1
                        
                        if self.is_valid_solution(temp, size_distance):
                            mutated = temp
                            mutation_happened = True
            
            # Strategy 3: Vehicle reduction - if possible, reduce total vehicles
            if random.random() < 0.3:  # 30% chance
                # Sort by range (highest first)
                range_ranking = sorted(vehicle_types, 
                                     key=lambda vt: vehicle_ranges[vt], 
                                     reverse=True)
                
                # Try to replace two low-range vehicles with one high-range vehicle
                for high_range_vt in range_ranking:
                    for low_range_vt in reversed(range_ranking):
                        if high_range_vt == low_range_vt or mutated[low_range_vt] < 2:
                            continue
                        
                        temp = mutated.copy()
                        temp[low_range_vt] -= 2
                        temp[high_range_vt] += 1
                        
                        if self.is_valid_solution(temp, size_distance):
                            mutated = temp
                            mutation_happened = True
                            break
                    
                    if mutation_happened:
                        break
            
            # If no mutation happened, force a simple change
            if not mutation_happened:
                vt = random.choice(vehicle_types)
                temp = mutated.copy()
                if random.random() < 0.5 and temp[vt] > 0:
                    temp[vt] -= 1
                else:
                    temp[vt] += 1
                
                if self.is_valid_solution(temp, size_distance):
                    mutated = temp
                    mutation_happened = True
            
            # Check if valid
            if self.is_valid_solution(mutated, size_distance):
                return mutated
            
            # Try to repair if invalid
            repaired = self._repair_solution(mutated, size_distance)
            if self.is_valid_solution(repaired, size_distance):
                return repaired
        
        # If all attempts fail, return original solution or a repaired version of it
        repaired_original = self._repair_solution(solution, size_distance)
        if self.is_valid_solution(repaired_original, size_distance):
            return repaired_original
        
        # Last resort: create a basic valid solution
        return self._create_basic_valid_solution(size_distance)

    def optimize(self, size_distance: Tuple, generations: int = 250) -> Dict:
        """Run genetic algorithm to find optimal fleet combination"""
        # Generate diverse initial population
        population_size = 150
        population = self.generate_initial_population(size_distance, population_size)
        
        best_solution = None
        best_fitness = float('-inf')
        no_improvement_count = 0
        
        # Track best solutions for comparison
        emissions_by_generation = []
        
        for gen in range(generations):
            # Calculate fitness for entire population
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            # Check if we have a new best solution
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
                no_improvement_count = 0
                
                # Calculate and save emissions for this generation
                best_emissions = self.calculate_emissions(best_solution, size_distance)
                emissions_by_generation.append(best_emissions)
            else:
                no_improvement_count += 1
            
            # Early stopping if no improvement for many generations
            if no_improvement_count >= 50:
                break
            
            # Select parents (use tournament selection)
            parents = []
            tournament_size = max(3, len(population) // 20)  # Dynamic tournament size
            for _ in range(len(population)):
                tournament = random.sample(fitness_scores, tournament_size)
                tournament.sort(key=lambda x: x[1], reverse=True)
                parents.append(tournament[0][0])
            
            # Create next generation
            next_generation = []
            
            # Elitism: keep top solutions
            elite_count = max(2, len(population) // 10)
            next_generation.extend([score[0] for score in fitness_scores[:elite_count]])
            
            # Fill the rest through crossover and mutation
            while len(next_generation) < population_size:
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = self.crossover(parent1, parent2, size_distance)
                
                # Apply mutation with varying rates based on generation
                # Higher mutation early, lower mutation later
                mutation_rate = 0.3 * (1 - min(1.0, gen / (generations * 0.7)))
                
                child1 = self.mutate(child1, size_distance, mutation_rate)
                child2 = self.mutate(child2, size_distance, mutation_rate)
                
                next_generation.append(child1)
                if len(next_generation) < population_size:
                    next_generation.append(child2)
            
            population = next_generation
        
        # Validate that best solution meets demand
        if best_solution and not self.is_valid_solution(best_solution, size_distance):
            best_solution = self._repair_solution(best_solution, size_distance)
            
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        """Run optimization and return results"""
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            print(f"Optimizing for Size: {size_distance[0]}, Distance: {size_distance[1]}...")
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            # Validate solution meets demand
            vehicles = self.vehicles_by_size_distance[size_distance] 
            demand = vehicles[0]['demand']
            total_capacity = sum(best_solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
            
            if total_capacity < demand:
                print(f"Warning: Solution does not meet demand for {size_distance}. Repairing...")
                best_solution = self._repair_solution(best_solution, size_distance)
                total_capacity = sum(best_solution[v['vehicle_type']] * v['yearly_range'] for v in vehicles)
            
            demand_fulfillment = self.calculate_demand_fulfillment(total_capacity, demand)
            size_distance_emissions = 0
            
            # Sort vehicles by emissions to allocate demand to lowest emission vehicles first
            sorted_vehicles = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
            remaining_demand = demand
            
            # Calculate emissions by allocating demand optimally
            for vehicle in sorted_vehicles:
                vehicle_type = vehicle['vehicle_type']
                num_vehicles = best_solution[vehicle_type]
                
                if num_vehicles > 0:
                    # Calculate how much this vehicle type can cover
                    vehicle_capacity = num_vehicles * vehicle['yearly_range']
                    assigned_demand = min(vehicle_capacity, remaining_demand)
                    remaining_demand -= assigned_demand
                    
                    # If no more demand to assign, vehicle is unused
                    if assigned_demand <= 0:
                        continue
                    
                    # Calculate emissions
                    emissions = vehicle['carbon_emissions_per_km'] * assigned_demand
                    size_distance_emissions += emissions
                    
                    # Calculate utilization for this vehicle type
                    utilization = self.calculate_utilization(
                        assigned_demand,
                        num_vehicles,
                        vehicle['yearly_range']
                    )
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": demand,
                        "Assigned Demand": round(assigned_demand, 2),
                        "Yearly Range": vehicle['yearly_range'],
                        "Utilization": round(utilization, 2),
                        "Demand_Fulfillment": round(demand_fulfillment, 2),
                        "Carbon Emissions": round(emissions, 2),
                        "Emissions Per KM": round(vehicle['carbon_emissions_per_km'], 4)
                    })
            
            total_carbon_emissions += size_distance_emissions
            
            # Verify all demand is allocated
            if abs(remaining_demand) > 0.01:  # Allow small floating-point error
                print(f"Warning: {remaining_demand:.2f} km of demand not allocated for {size_distance}")

        print(f"\nTotal Carbon Emissions: {total_carbon_emissions:.2f}")
        return pd.DataFrame(results)

def main(csv_path: str):
    """Main function to run the optimization"""
    print("\nRunning optimization with pure carbon emissions minimization...")
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print("\nOptimized Fleet Allocation:")
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)